In [3]:
import sys
import numpy as np
import time
import pickle
import pandas as pd
import copy
import matplotlib.pyplot as plt
import pandas as pd

import allensdk
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

##### Due to the large file size, the files associated with this notebook is not yet uploaded on github. Only the notebook is frequently updated. To be resolved later

# Mouse Connectivity
#### Author: Gisu David Ham. Davidnh8@gmail.com
This notebook analyzes the mouse connectivity data for the purpose of calculating the information flow in the brain. We reply on AllenSDK with some custom codes.

As a brief summary, Allen brain Atlas: Mouse Brain Connectivity contains data on  experiments on the mouse brain. Each experiment has an associated (unique) injection site where green fluroescent protein virus is injected. This injection infects the cell-bodies of the neurons in the injection site, and they propagates along the axons (carrier of the action potential of neurons) where they reach its terminal and emit green lights. These signals then are imaged through green-flurescent imaging and mapped accordingly. More information can be found on: http://connectivity.brain-map.org/. Some defintions relevant to the throughout the notbook are defined as follows.

projection density = sum of detected pixels / sum of all pixels in division  
projection energy = sum of detected pixel intensity / sum of all pixels in division  
injection_fraction = fraction of pixels belonging to manually annotated injection site  
injection_density = density of detected pixels within the manually annotated injection site  
injection_energy = energy of detected pixels within the manually annotated injection site  
data_mask = binary mask indicating if a voxel contains valid data (0=invalid, 1=valid). Only valid voxels should be used for analysis  


This notebook carefully combines multiple experiments to create a connectome data. This data is then traslated into the information flow data, namely we determine how much information flows at a regional level.

## Fetch all the Experiment list


First we create MouseConnectivityCache instance.

Fome this we may fetch the experiment list. One in dataFrame and one in dictionary format.

In [4]:
# MouseConnectivityCache allows access to data stored within
mcc=MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')

#get the id of all experiments.
#Create one for Pandas dataframe and the other in terms of dictionary
experiment_list = mcc.get_experiments(dataframe=True)
experiment_list_dict=mcc.get_experiments(dataframe=False)

Sample of some of the entries of the experiment_list looks like:

In [5]:
experiment_list[:3]

,gender,id,injection_structures,injection_volume,injection_x,injection_y,injection_z,primary_injection_structure,product_id,specimen_name,strain,structure_abbrev,structure_id,structure_name,transgenic_line,transgenic_line_id
id,,,,,,,,,,,,,,,,
527712447,F,527712447,"[502, 926, 1084, 484682470]",0.006655,9240,3070,8990,502,5,Penk-IRES2-Cre-neo-249961,C57BL/6J,SUB,502,Subiculum,Penk-IRES2-Cre-neo,298725927.0
301875966,M,301875966,"[574, 931]",0.105746,9170,6850,6200,574,5,Gabrr3-Cre_KC112-3467,C57BL/6J,PG,931,Pontine gray,Gabrr3-Cre_KC112,177838877.0
520336173,M,520336173,"[1, 210, 491, 525, 1004]",0.025762,7810,6550,6450,1,5,Hdc-Cre_IM1-204103,None,TMv,1,"Tuberomammillary nucleus, ventral part",Hdc-Cre_IM1,177839494.0


where the columns are:

In [6]:
experiment_list.columns

Index([                     u'gender',                          u'id',
              u'injection_structures',            u'injection_volume',
                       u'injection_x',                 u'injection_y',
                       u'injection_z', u'primary_injection_structure',
                        u'product_id',               u'specimen_name',
                            u'strain',            u'structure_abbrev',
                      u'structure_id',              u'structure_name',
                   u'transgenic_line',          u'transgenic_line_id'],
      dtype='object')

with the number of separate experiments being:

In [7]:
len(experiment_list_dict)

2995

This is for allensdk 0.14.5. Number of experiments may vary depending on the version of the package. Note that each injection sites has a unique experiemnt, that is, no more than one injection sites were allowed in a single mouse brain.

## Annotation Volume

We then download annotation volume for future uses. This comes in the form of an array where structures are denoted by the values of the elements. Spatial coordinates on the other hand, are represented by the indices of the array. First create an instance using the MouseConnectivityCache (mcc)


In [8]:
annt=mcc.get_annotation_volume()[0]

the annotation volume has the shape

In [9]:
annt.shape

(528L, 320L, 456L)

for future, we collect the id of all the structures present in the annt volume

In [10]:
inp=raw_input("computationally intensive. Run anyway? (y/n)")

if inp=="y":
    annt_structures=[]
    for i in range(annt.shape[0]):
        for j in range(annt.shape[1]):
            for k in range(annt.shape[2]):
                if annt[i,j,k] not in annt_structures:
                    annt_structures.append(annt[i,j,k])
else:
    None

computationally intensive. Run anyway? (y/n)n


## Structure_tree

Allenmouse structure ontology is organized as a tree where each edge indicates the physical containment. Structuretree can be used to fetch properties of a given structure, including its id, name, acronym, etc..

In [10]:
structure_tree=mcc.get_structure_tree()

sample of each structure is shown:

In [11]:
pd.DataFrame(structure_tree.get_structures_by_name(['Thalamus','Hypothalamus']))

,acronym,graph_id,graph_order,id,name,rgb_triplet,structure_id_path,structure_set_ids
0,TH,1,641,549,Thalamus,"[255, 112, 128]","[997, 8, 343, 1129, 549]","[2, 112905828, 691663206, 12, 184527634, 11290..."
1,HY,1,715,1097,Hypothalamus,"[230, 68, 56]","[997, 8, 343, 1129, 1097]","[2, 112905828, 691663206, 12, 184527634, 11290..."


where the columns are:

In [12]:
for i in structure_tree.get_structures_by_name(['Thalamus'])[0]:
    print(i)

rgb_triplet
graph_id
name
acronym
graph_order
structure_id_path
structure_set_ids
id


## structure set

On top of strucure tree, AllenmouseSDK provides a convenient way to group structures together through the structure sets.
For example, if i wish to fetch structures trees beonging to Midbrain, I can easily do so.

In [13]:
from allensdk.api.queries.ontologies_api import OntologiesApi

#initialize oapi_inst instance
oapi_inst=OntologiesApi()

In [14]:
structure_set_ids=structure_tree.get_structure_sets()
structure_set=pd.DataFrame(oapi_inst.get_structure_sets(structure_set_ids))
structure_set

,description,id,name
0,List of structures in Isocortex layer 5,667481446,Isocortex layer 5
1,List of structures in Isocortex layer 6b,667481450,Isocortex layer 6b
2,Summary structures of the cerebellum,688152368,Cerebellum
3,List of structures for ABA Differential Search,12,ABA - Differential Search
4,List of valid structures for projection target...,184527634,Mouse Connectivity - Target Search
5,Structures whose surfaces are represented by a...,691663206,Mouse Brain - Has Surface Mesh
6,Summary structures of the midbrain,688152365,Midbrain
7,Summary structures of the medulla,688152367,Medulla
8,Summary structures of the striatum,688152361,Striatum
9,Structures representing subdivisions of the mo...,687527945,Mouse Connectivity - Summary


As shown above, we can select the structure set based on its description and fetch the desired collection of structures>

Here our interest is in the projection research, therefore we fetch the summary structures from 184527634 (Mouse Connectivity - Target Search)

In [15]:
projection_structures_target=pd.DataFrame(structure_tree.get_structures_by_set_id([184527634]))

### how many structures in each?

Out of curiosity, we explore how many structures are contained in each structure set:

In [16]:
#very clever name for a variable indicating exactly what it is used for.
printme={'experiment id':[],'number of structures':[]}

for i in structure_set[structure_set.columns[1]]:  #column 1 is id
    printme['experiment id'].append(i)
    printme['number of structures'].append(len(structure_tree.get_structures_by_set_id([i])))
pd.DataFrame(printme)

,experiment id,number of structures
0,667481446,43
1,667481450,43
2,688152368,18
3,12,843
4,184527634,840
5,691663206,840
6,688152365,39
7,688152367,45
8,688152361,14
9,687527945,293


# Maps

Each structure can be denoted by all kinds of variables such as its id, acronym, or its full name. Therefore it is convenient to define maps that will take care of this. We want the map to function as a converter between variables.


In [17]:
projection_structures_inj=pd.DataFrame(projection_structures_target)

acronym=projection_structures_inj['acronym'].values.tolist()
acronym_to_id={}
acronym_to_name={}
acronym_to_volume={}
id_to_idpath={}
id_to_name={}

for i in range(len(acronym)):
    a=projection_structures_inj['acronym'][i] #structure acronym
    b=projection_structures_inj['structure_id_path'][i][-1] #sturcture id
    c=projection_structures_inj['name'][i] #structure name
    d=projection_structures_inj['structure_id_path'][i] #structure id path
    acronym_to_id[a]=b
    acronym_to_name[a]=c
    id_to_name[b]=c
    id_to_idpath[b]=d


define inverse map for the dictionary

In [18]:
def invert_me(arg):
    """inverts a map
        args:
            arg(dictionary): input dictionary
        returns:
            inverted dictionary"""
    return {v: k for k,v in arg.iteritems()}

Then simply invert the deried dictionaries:

In [19]:
id_to_acronym=invert_me(acronym_to_id)
name_to_acronym=invert_me(acronym_to_name)
name_to_id=invert_me(id_to_name)

Since the structures are organized as trees, it would be nice to have a dictionary that can return all its daughter structures (or all structures beneath it). Hence here we define a dictionary that contains its direct substructures (only one level below) called Substr

In [20]:
projection_structures_inj.columns[-2]

'structure_id_path'

In [21]:
len(acronym)

840

We fetch the information about the hierachy based on the structure_id_path

In [22]:
Substr={}
Substr[997]=[]
for acr in acronym:
    Substr[acronym_to_id[acr]]=[]

for i in projection_structures_inj['structure_id_path']:
    try:
        Substr[i[-2]].append(i[-1])
    except:
        print i

[997, 73, 81, 98]
[997, 73, 81, 108]


In [23]:
print id_to_name[98], id_to_name[108]

subependymal zone choroid plexus


Even though the structure_id_path of the chosen structure set contains 73 and 81, they are however not contained in the actual structure list of the structure set (possible error?). Removing those regions from the study would not have any significant impact

structure with ID 81 has no data in projection_structures_inj structure tree. 

Now we define functions that unpacks a list of all structures below a particular structure using Substr

In [24]:
def Unpack(structure_id,MST,include_self=False):
    """returns a list of substructures of a given structure
        args:
            structures_id(int): id of the structure
            MST(dictionary): My Structure Tree
            include_self(bool): decide whether to include the initial structure, default=False
        
        returns:
            king(list) """
    
    #to be returned at the end of everything
    unpacked=[]
    
    # apply recursion and add structures to unpacked 
    # until the length of each substructure becomes zero
    def Unpack_(structure_id,MST):
        """Primarily used for 'recursion' of the unpack"""
        
        leng=len(MST[structure_id])

        if leng>=1:
            unpacked.append(structure_id)
            for i in MST[structure_id]:
                Unpack_(i,MST)

        elif leng==0:
            unpacked.append(structure_id)
        

    Unpack_(structure_id,MST)
    
    # remove self if include_self false
    if include_self:
        pass
    elif not include_self:
        unpacked.remove(structure_id)
    
    return unpacked

There is just something about the function above that I do not like. Perhaps I should try writing it as a class later.

In [44]:
name_to_id['Thalamus']

549

In [45]:
len(Unpack(997,Substr,include_self=False))

838

## Injection Fraction (currently not needed)

We wish to find how many fractions of the total injection volume of an experiment belong to the structure of interest.

id_to_idpath defined earlier is not complete without structure id 997 and 0

In [65]:
id_to_idpath[0]=[]
id_to_idpath[997]=[997]

structure_id_list_exception=[81]

#### Below functions no longer needed for my project

In [66]:
def create_structure_dictionary(annt_values,inj_density_values,include_density=True):
    """args:
        annt_values: (array, int 32): array of structure ids obtained from annotation volume[injection_mask]
        inj_density_values (array): 
        
        returns:
            structuer_dict(dictionary): a map from a structure id to its fraction in the injection volume
            """
    
    #check that annt_values and inj_density are the same size
    if len(annt_values)!=len(inj_density_values):
        raise ValueError('annt volume and inj density must be the same size')
    
    structure_dict={}
    leng=len(annt_values)
    exception_count=0
    exception_id_list=[]
    
    # include density
    if include_density:
        
        for i in range(leng):
            
            if annt_values[i] in structure_id_list_exception:
                exception_id_list.append(annt)
                pass
            elif annt_values[i] not in structure_id_list_exception:
                if annt_values[i] not in structure_dict:
                    structure_dict[annt_values[i]]=0
                elif annt_values[i] in structure_dict:
                    structure_dict[annt_values[i]]+=inj_density_values[i]
                    
                    
        #need to add the id of each superstructures of structure_dict
        extra=[]
        
        for i in structure_dict:
            try:
                sup=id_to_id_path[i]
                for j in sup:
                    if j not in structure_dict:
                        extra.append(j)
            except:
                exception_count+=1
                pass
        for i in extra:
            structure_dict[i]=0
                
        return structure_dict,exception_count,exception_id_list
    
    
    elif not include_density:
        for i in arg:
            try:
                if i not in structure_dict:
                    structure_dict[i]=0
            except:
                exception_count+=1
                pass

            if i in structure_dict:
                structure_dict[i]+=1
        return structure_dict,exception_count,exception_id_list
    
def generate_experiment_structure_fractions( experiment_list, annt,MST,include_density=True):
    """returns a dictionary that contains fraction information about the injection volume of the structure"""
    
    """args:
        experiment_list(list): list of experiment ids
        annt (ndarray) annotation volume
        include_density (bool): decide whether to calculate using the density of simple volume number.
        Default=True"""
    
    #returning dictionary
    return_dict={}
    
    #iterate over the given experiment list.
    for i in experiment_list:
        
        #fetch injection density information
        inj_density=mcc.get_injection_density(i)[0]
        #create booleean array that is True on the injection sites
        inj_density_bool=inj_density!=0.0
        
        #np.where returns a tuple (x,y,z) for 3d, which are to be unpacked immediately
        a,b,c=np.where(inj_density_bool)
        #using the voxel coordinates a,b,c, find density values at each coordinate. 
        inj_density_values=inj_density[a,b,c]
        #and annotation values that contain id list
        annt_values=annt[a,b,c]
        
        
        
        #call the function create_structure_dictionary that analyzies the fraction information
        sub_dict,exception_count_sub,exception_id_list_sub=create_structure_dictionary(annt_values,inj_density_values,include_density=True)
        
        
        
        for p in sub_dict:
            if not include_density:
                leng=annt_values.shape[0]
                sub_dict[p]=float(sub_dict[p])/leng
            elif include_density:
                #normalize the fraction
                sub_dict[p]=float(sub_dict[p])/sum(inj_density_values)
        
        return_dict[i]=sub_dict
    
    exception_count=0
    exception_id_list=[]
    
    # now add the fraction of substructures to the superstructure
    #normal copy would not really 'copy' the dictionary. Thus use copy.deepcopy
    return_dict_modified=copy.deepcopy(return_dict)
    for i in return_dict:
        for j in return_dict[i]:
            #call unpack function to list all the substructures
            if j in projection_structures_inj:
                unpackedpre=Unpack(j,MST,include_self=True)

                for k in unpackedpre:
                    if j!=k:
                        try:
                            aa=return_dict[i][k].copy()
                            return_dict_modified[i][j]+=aa
                        except:
                            exception_count+=1
                            pass
            else:
                exception_count+=1
                exception_id_list.append(j)
                pass
    return return_dict_modified,exception_count,exception_id_list,exception_count_sub,exception_id_list_sub

def map_to_acronym(dict_):
    return_dict={}
    exception_count,exception_id=(0,[])
    for i in dict_:
        try:
            return_dict[id_to_acronym[i]]=dict_[i]
        except:
            exception_count+1
            exception_id.append(i)
            pass
    return return_dict


In [67]:
run=raw_input("Computationally intensive (roughly 6-8 hours) Run? (y/n)")
if run=="y":
    Master_injection_fraction_dictionary={}
    ec=0
    ecl=[]

    for i in experiment_list['id'].tolist():
        ecp,ec,ecl,ecs,ecls=generate_experiment_structure_fractions([i],annt,Substr,include_density=True)
        Master_injection_fraction_dictionary[i]=ecp[i]
        
elif run=="n":
    print "operation cancelled"


Computationally intensive (roughly 6-8 hours) Run? (y/n)n
operation cancelled


### Save & Load station
save or load Master_injection_fraction_dictionary

In [40]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("Master_injection_fraction_dictionary",'wb') as f:
        pickle.dump(Master_injection_fraction_dictionary,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [25]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    Master_injection_fraction_dictionary={}
    with open("Master_injection_fraction_dictionary",'r') as f:
        Master_injection_fraction_dictionary=pickle.load(f)
elif run=="n":
    print "operation cancelled"

Load? (y/n)y


# Get Structure Volume (Currently Not needed)

In [72]:
def volume_dictionary_gen(acr_list):
    """based on input acronym list, return a dictionary that maps ID to its volume.
    
        This is a poor workaround the fact that I don't know how to import the volume data of the structure from the website.
        I can only calculate the volume from the annotation files, but there are discrepencies between that and the unionization
        records.
    
    args:
        acr_list (list): input acronym list
        
    returns:
        returndict(dictionary): map from id to its volume
        exception_structure_acr (list) a list of structure that do not have the volume data available from this method"""
    
    returndict={}
    exception_struct_acr=[]
    
    for i in acr_list:
        
        '''volume data is acquired through unionization.
            From the experiment lists of a structure, I target itself and return its unionization records.
            '''
        struct=structure_tree.get_structures_by_acronym([i])[0]
        struct_experiments=mcc.get_experiments(cre=None,
                                              injection_structure_ids=[struct['id']])
        
        
        '''If error, then it must be the case that the structure does not belong to any injection site.
            I believe it is still possible for other structures to project to these regions, but for this particular analysis,
            I ignore these structures.'''
        try:
            structure_unionizes=mcc.get_structure_unionizes( [e['id'] for e in struct_experiments],
                                                          is_injection=None,
                                                          structure_ids=[struct['id']], # all
                                                          include_descendants=True,
                                                           hemisphere_ids=[2])
        except:
            exception_struct_acr.append(i)
            pass
        
        for j in range(len(structure_unionizes)):
            a=structure_unionizes['structure_id'][j]
            b=structure_unionizes['volume'][j]

            if a not in returndict:
                returndict[a]=b

    return returndict,exception_struct_acr

In [71]:
run=raw_input("computationally demanding. run? (y/n)")

if run=="y":

    id_to_volume={}
    str_id=structure_unionizes['structure_id']
    str_vol=structure_unionizes['volume']
    for i in range(len(str_id)):
        if i not in id_to_volume.keys():
            id_to_volume[str_id[i]]=str_vol[i]
elif run =="n":
    print "operation cancelled"

computationally demanding. run? (y/n)n
operation cancelled


### Save and Load
save and load id_to_volume dictionary.

In [49]:
"""SAVE"""
run=raw_input("Save? (y/n)")

if run=="y":
    with open ("id_to_volume",'wb') as f:
        pickle.dump(id_to_volume,f)
    
elif run=="n":
    print "operation cancelled"

Save? (y/n)y


In [ ]:
"""Load"""
run=raw_input("Load? (y/n)")

if run=="y":
    with open("id_to_volume",'r') as f:
        id_to_volume=pickle.load(f)
elif run=="n":
    print "operation cancelled"

### Sum proj (No longer needed)
Sum projection volumes and calculate information flow

In [74]:
def sum_proj(inj,target,hemisphere=[2],r1=0.1,r2=1.0):
    """
    args:
        inj (str): acronmy of the injection site
        target (str): acronym of the target structures
    
    returns:
    """
    
    #if not reciprocal:
    #    pass
    #elif reciprocal:
    #    save=inj
    #    inj=target
    #    target=save
        
    #1. get projection search results
    sinj=structure_tree.get_structures_by_acronym([inj])[0]
    starget=structure_tree.get_structures_by_acronym([target])[0]
    
    sinj_experiments=mcc.get_experiments(cre=None,
                                         injection_structure_ids=[sinj['id']])
    
    
    
    if len(sinj_experiments)==0:
        raise ValueError( 'No experiments exist for this structure')
        
    unionizes=mcc.get_structure_unionizes([ep['id'] for ep in sinj_experiments],
                                         is_injection=False,
                                         structure_ids=[starget['id']],
                                         include_descendants=False,
                                         hemisphere_ids=hemisphere)
    exception_count=0.0
    
    #2. using unionized records, get experiments id and its associated projection volume
    # from sinj to target
    a=unionizes['experiment_id']
    b=unionizes['projection_volume']
    
    #3. try fetching the volume data for the injection structure. return error otherwise
    try:
        sinj_structure_vol=float(id_to_volume[acronym_to_id[inj]])/2
    except:
        raise ValueError('Volume data does not exist for this structure')
    
    _error,__error,___error=0.0,0.0,0.0
    
    proj_sum=0.0
    tot_injection_volume_fraction=0.0
    
    for i in range(len(a)):
        p=i
        
        if float(expid_to_inj_volume_int[a[p]])>=r1 and float(expid_to_inj_volume_int[a[p]])<=r2:
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            #c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1
            
            c3=float(expid_to_inj_volume_int[a[p]])
            #c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3
            
    if tot_injection_volume_fraction>=sinj_structure_vol:
        #print "tot inj volume larger"
        #print "tot injection fraction",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        #print "tot inj volume smaller"
        #print "tot inj volume",tot_injection_volume_fraction
        #print "structure vol",sinj_structure_vol
        proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
        ___error="NA"
        #___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
        return proj_sum,___error
        
        
            
""" 
            #proj sum = proj volume * fraction of the sinj region
            c1=float(b[p])
            c2=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            proj_sum+=c1*c2
            _error+=c1*c2*np.sqrt( (0.10)**2 + (0.02)**2)
            
            #tot inj_vol_fraction= injection volume of exp * fraction of the sinj region
            
            c3=float(expid_to_inj_volume_int[a[p]])
            c4=float(Master_injection_fraction_dictionary[a[p]][acronym_to_id[inj]])
            tot_injection_volume_fraction+=c3*c4
            __error+=c3*c4*np.sqrt((0.05)**2 + (0.02)**2)
        else:
            pass
    if tot_injection_volume_fraction>=sinj_structure_vol:
        try:
            print "tot inj volume larger"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*tot_injection_volume_fraction/sinj_structure_vol
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        except:
            print 'exception occured'
    elif tot_injection_volume_fraction<=sinj_structure_vol:
        try:
            print "tot inj volume smaller"
            print tot_injection_volume_fraction
            print sinj_structure_vol
            proj_sum=92000*proj_sum*sinj_structure_vol/tot_injection_volume_fraction
            ___error=(proj_sum*sinj_structure_vol/tot_injection_volume_fraction)*np.sqrt((0.10)**2+(__error/tot_injection_volume_fraction)**2+(_error/proj_sum)**2)
            return proj_sum,___error
        
        except ZeroDivisionError:
            return 0,0,0,0
"""
None

In [75]:
#create list of structures
Cortex_list=[id_to_acronym[k] for k in Substr[acronym_to_id['Isocortex']]]
BS_nuclei=['SCs','SCm','PB','NTS','PAG']
HY_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['HY']]]
TH_nuclei=[id_to_acronym[k] for k in Substr[acronym_to_id['TH']]]

In [398]:
for i in CTX_to_BS:
    print i,round(sum([CTX_to_BS[i][j] for j in CTX_to_BS[i]]),2)

VISC 448146.48
SS 1927754.04
ACA 3959033.84
AUD 414137.81
ILA 869132.09
AI 77975.13
TEa 0.0
MO 2649989.21
GU 1539.87
ECT 137089.71
VIS 10282252.17
FRP 10576.86
PTLp 147662.07
ORB 978561.55
RSP 471877.36
PL 238919.97


In [172]:
ab={'a':[1,2],'b':[3,4]}
c=pd.DataFrame(ab)
c

,a,b
0,1,3
1,2,4


In [170]:
Voxel_connectivity

ipsi                                                         \
                 FRP           MOp           MOs        SSp-n       SSp-bfd   
FRP       895.510252   1077.369403   3464.226521    60.069171     32.133350   
MOp       698.340185  30667.317616  20301.732190  1732.338470   1107.394282   
MOs      3049.925480  19919.693919  37126.489780  1743.149433   2850.520739   
SSp-n     269.194751   4402.052803   6196.124305  7808.409586   1444.595067   
SSp-bfd    86.466223   3494.876434   6286.566123  2221.830052  18647.612604   
SSp-ll      4.555590   2587.996071    850.381134    69.325264    649.186424   
SSp-m     341.500125  13809.886944   9986.036410  4371.886112    156.942340   
SSp-ul    102.562584   6714.643757   3345.208386  1849.982256   1391.760906   
SSp-tr      2.421157    939.311607    673.175352    34.513232   1346.795406   
SSp-un     65.794457   2036.370141   1734.126990  1767.636266   1293.003092   
SSs       290.518579   8817.235987   6745.161469  5559.471182   6774.078372   
GU         61.839021   1201.662205    849.484489    59.554753     26.190293   
VISC      120.183732   1381.696399   1368.740048   428.261642    263.962420   
AUDd        0.714349    101.874328    188.694990    15.722669    561.879368   
AUDp        0.571729     89.944548    177.253114     7.986387    394.085028   
AUDpo       0.312581     36.183566     97.920138     1.930578    160.356511   
AUDv        6.879802    128.671022    173.543370    66.334677    280.563337   
VISal       0.441047     64.220659    211.015163     2.503271    374.981855   
VISam       0.695047    132.939703    504.856113     3.571251    322.015749   
VISl        0.771871     31.706222    158.914843     1.771709    172.991546   
VISp        0.495857     50.727830    276.557538     5.826796    406.724407   
VISpl       0.356917      7.973756     40.840335     1.260392     58.089321   
VISpm       0.408625     78.100278    313.994438     3.724421    176.171966   
VISli       0.687960     20.945105     77.007225     1.437753     94.374049   
VISpor      0.879363     22.473876     85.812534     1.987131    114.258838   
ACAd      141.163569   1384.710019   6270.774801    35.049808    746.054357   
ACAv       30.652898    229.592291   2287.001157     2.630478    127.616633   
PL        341.346186    827.445239   3053.485726    22.172251    171.115002   
ILA        34.821682     59.954389    396.635024     3.538801     48.723864   
ORBl     1064.771001    987.852832   4135.106892   120.125505    276.468942   
...              ...           ...           ...          ...           ...   
NR          0.003576      0.050502      0.034683     0.033099      0.037655   
PRP         0.003280      0.191759      0.129460     0.139673      0.078672   
PPY         0.001490      0.113352      0.100305     0.143808      0.062309   
LAV         0.008698      0.276711      0.093815     0.421883      0.162138   
MV          0.085846      1.549621      1.257492     1.590396      0.795344   
SPIV        0.007234      0.349060      0.113849     0.711805      0.376393   
SUV         0.015466      0.287070      0.154348     0.294924      0.152015   
x           0.000562      0.027217      0.007658     0.043273      0.024390   
XII         0.025111      0.516441      0.303008     0.506084      0.597655   
y           0.000505      0.015230      0.003397     0.025010      0.011199   
RM          0.004253      0.134539      0.133501     0.113448      0.072677   
RPA         0.001932      0.090873      0.078996     0.112952      0.057981   
RO          0.003391      0.100142      0.080385     0.084870      0.051974   
LING        0.016345      0.251453      0.245536     0.433227      0.242459   
CENT        0.258875      4.115568      3.135022     8.060353      3.551480   
CUL         0.228882      8.225923      3.003876    11.400271      5.833175   
DEC         0.014336      2.680004      1.329060     2.802036      0.731568   
FOTU        0.000069      0.032322      0.007330     0.019789     

## Mcmodels
Here we use Mcmodels and their available data to explore the connectivity strength and the information flow. We import some of the packages again to make this section more standlone. Aside from using Substr and the maps, this section is completely independent of the previous contents.

Structure sets used by Knox, et al on 'High resolution data_diven model' uses intermediate level summary structures. This most likely corresponds to the structure set number 687527945 'Structures representing the subdivisions of the Mouse Brain'

In [25]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
mcc=MouseConnectivityCache(manifest_file='connectivity/mouse_connectivity_manifest.json')
structure_tree=mcc.get_structure_tree()

#new packages to import
from mcmodels.core import VoxelModelCache
Voxel_cache = VoxelModelCache()
Voxel_connectivity=Voxel_cache.get_connection_strength()
Voxel_connectivity_dict=dict(Voxel_connectivity)

Compare the structure set 687527945 and the voxel structure set

In [26]:

structure687527945=list(pd.DataFrame(structure_tree.get_structures_by_set_id([687527945]))['acronym'])
Voxel_Structures=set([i[1] for i in Voxel_connectivity_dict.keys()])

In [27]:
print len(structure687527945), len(Voxel_Structures)

293 291


the former has 2 more structures, which are:

In [28]:
for i in structure687527945:
    if i not in Voxel_Structures:
        print i

ENTmv
fiber tracts


In [29]:
acronym_to_name['ENTmv']

u'Entorhinal area, medial part, ventral zone'

https://en.wikipedia.org/wiki/Entorhinal_cortex
The entorhinal cortex (EC) (ento = interior, rhino = nose, entorhinal = interior to the rhinal sulcus) is an area of the brain located in the medial temporal lobe and functioning as a hub in a widespread network for memory, navigation and the perception of time.[1] The EC is the main interface between the hippocampus and neocortex.

Ignoring this small structure would not affect the entire analysis.

## Exploring Damasio Network

This section contains work studying the network that are vital for consciousness by Damasio in his book : https://www.goodreads.com/book/show/7766914-self-comes-to-mind

Brain stem is divided into:

In [30]:
BSsub=[id_to_name[i] for i in Substr[acronym_to_id['BS']]]
BSsub

[u'Midbrain', u'Hindbrain', u'Interbrain']

where each can further be subdivided into:

In [31]:
MBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[0]]]]
MBsub

[u'Midbrain, motor related',
 u'Midbrain, sensory related',
 u'Midbrain, behavioral state related']

In [32]:
HBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[1]]]]
HBsub

[u'Medulla', u'Pons']

In [33]:
IBsub=[id_to_name[i] for i in Substr[name_to_id[BSsub[2]]]]
IBsub

[u'Hypothalamus', u'Thalamus']

but of course non of them are included in the Voxel_Structures, but some of their sub(-sub)structures are definitely included. The question is, which ones? We use tree to visualize this first

In [34]:
from anytree import Node, RenderTree

In [35]:
import string
def print_code_for_tree(acr):
    """"""
    #returns list of substructures - outputs are in terms of structure id
    substr=[id_to_acronym[i] for i in Substr[acronym_to_id[acr]]]
    
    #variable names cannot contain - and /. replace temporarily
    if "-" in acr or "/" in acr:
        acr=string.replace(acr,"-","_")
        acr=string.replace(acr,"/","and")
    
    if len(substr)>0:
        for j in substr:
            #exp = mcc.get_experiments(file_name='experiments_for_'+j+'.json',
            #                      injection_structure_ids=[acronym_to_id[j]])
            #n_exp=len(exp)
            n_exp='NA'
            
            #check whether the structure is in Voxel_Structures
            if j in Voxel_Structures:
                inVoxel=True
            else:
                inVoxel=False
                    
            #python variable name does not allow - or /. Replace with _ if exists
            if ("-" in j) or ("/" in j):
                
                #check whether the structure is in Voxel_Structures
                j=string.replace(j,"-","_")
                j=string.replace(j,"/","and")

                print str(j)+" = "+"Node('"+ j+" inVoxel="+str(inVoxel) + "  n_exp= "+str(n_exp) +"', parent="+acr+")"
                
                #make sure to return j to its original since otherwise Substr
                #will not recognize its altered form.
                j=string.replace(j,"_","-")
                j=string.replace(j,"and","/")
                
                """RECURSION"""
                print_code_for_tree(j)
                
            else:
                print str(j)+" = "+"Node('"+ j +" inVoxel="+str(inVoxel)+ "  n_exp= "+str(n_exp) +"', parent="+acr+")"
                
                
                """RECURSION"""
                print_code_for_tree(j)
    else:
        None

In [34]:
print_code_for_tree('BS')

MB = Node('MB inVoxel=False  n_exp= NA', parent=BS)
MBmot = Node('MBmot inVoxel=False  n_exp= NA', parent=MB)
III = Node('III inVoxel=True  n_exp= NA', parent=MBmot)
MT = Node('MT inVoxel=False  n_exp= NA', parent=MBmot)
LT = Node('LT inVoxel=True  n_exp= NA', parent=MBmot)
DT = Node('DT inVoxel=False  n_exp= NA', parent=MBmot)
IV = Node('IV inVoxel=True  n_exp= NA', parent=MBmot)
MRN = Node('MRN inVoxel=True  n_exp= NA', parent=MBmot)
PN = Node('PN inVoxel=False  n_exp= NA', parent=MBmot)
RN = Node('RN inVoxel=True  n_exp= NA', parent=MBmot)
AT = Node('AT inVoxel=True  n_exp= NA', parent=MBmot)
RR = Node('RR inVoxel=True  n_exp= NA', parent=MBmot)
SCm = Node('SCm inVoxel=True  n_exp= NA', parent=MBmot)
SCig = Node('SCig inVoxel=False  n_exp= NA', parent=SCm)
SCig_a = Node('SCig_a inVoxel=False  n_exp= NA', parent=SCig)
SCig_b = Node('SCig_b inVoxel=False  n_exp= NA', parent=SCig)
SCig_c = Node('SCig_c inVoxel=False  n_exp= NA', parent=SCig)
SCiw = Node('SCiw inVoxel=False  n_exp= NA',

In [35]:
BS = Node('BS inVoxel=False')
CTX= Node('CTX inVoxel=False')

In [41]:
MB = Node('MB inVoxel=False  n_exp= NA', parent=BS)
MBmot = Node('MBmot inVoxel=False  n_exp= NA', parent=MB)
III = Node('III inVoxel=True  n_exp= NA', parent=MBmot)
MT = Node('MT inVoxel=False  n_exp= NA', parent=MBmot)
LT = Node('LT inVoxel=True  n_exp= NA', parent=MBmot)
DT = Node('DT inVoxel=False  n_exp= NA', parent=MBmot)
IV = Node('IV inVoxel=True  n_exp= NA', parent=MBmot)
MRN = Node('MRN inVoxel=True  n_exp= NA', parent=MBmot)
PN = Node('PN inVoxel=False  n_exp= NA', parent=MBmot)
RN = Node('RN inVoxel=True  n_exp= NA', parent=MBmot)
AT = Node('AT inVoxel=True  n_exp= NA', parent=MBmot)
RR = Node('RR inVoxel=True  n_exp= NA', parent=MBmot)
SCm = Node('SCm inVoxel=True  n_exp= NA', parent=MBmot)
SCig = Node('SCig inVoxel=False  n_exp= NA', parent=SCm)
SCig_a = Node('SCig_a inVoxel=False  n_exp= NA', parent=SCig)
SCig_b = Node('SCig_b inVoxel=False  n_exp= NA', parent=SCig)
SCig_c = Node('SCig_c inVoxel=False  n_exp= NA', parent=SCig)
SCiw = Node('SCiw inVoxel=False  n_exp= NA', parent=SCm)
SCdg = Node('SCdg inVoxel=False  n_exp= NA', parent=SCm)
SCdw = Node('SCdw inVoxel=False  n_exp= NA', parent=SCm)
CUN = Node('CUN inVoxel=True  n_exp= NA', parent=MBmot)
VTA = Node('VTA inVoxel=True  n_exp= NA', parent=MBmot)
VTN = Node('VTN inVoxel=True  n_exp= NA', parent=MBmot)
PAG = Node('PAG inVoxel=True  n_exp= NA', parent=MBmot)
PRC = Node('PRC inVoxel=False  n_exp= NA', parent=PAG)
INC = Node('INC inVoxel=False  n_exp= NA', parent=PAG)
ND = Node('ND inVoxel=False  n_exp= NA', parent=PAG)
Su3 = Node('Su3 inVoxel=False  n_exp= NA', parent=PAG)
EW = Node('EW inVoxel=True  n_exp= NA', parent=MBmot)
Pa4 = Node('Pa4 inVoxel=False  n_exp= NA', parent=MBmot)
PRT = Node('PRT inVoxel=False  n_exp= NA', parent=MBmot)
APN = Node('APN inVoxel=True  n_exp= NA', parent=PRT)
MPT = Node('MPT inVoxel=True  n_exp= NA', parent=PRT)
NOT = Node('NOT inVoxel=True  n_exp= NA', parent=PRT)
NPC = Node('NPC inVoxel=True  n_exp= NA', parent=PRT)
OP = Node('OP inVoxel=True  n_exp= NA', parent=PRT)
PPT = Node('PPT inVoxel=True  n_exp= NA', parent=PRT)
RPF = Node('RPF inVoxel=False  n_exp= NA', parent=PRT)
SNr = Node('SNr inVoxel=True  n_exp= NA', parent=MBmot)
MA3 = Node('MA3 inVoxel=False  n_exp= NA', parent=MBmot)
MBsen = Node('MBsen inVoxel=False  n_exp= NA', parent=MB)
IC = Node('IC inVoxel=True  n_exp= NA', parent=MBsen)
ICc = Node('ICc inVoxel=False  n_exp= NA', parent=IC)
ICd = Node('ICd inVoxel=False  n_exp= NA', parent=IC)
ICe = Node('ICe inVoxel=False  n_exp= NA', parent=IC)
SAG = Node('SAG inVoxel=True  n_exp= NA', parent=MBsen)
SCs = Node('SCs inVoxel=True  n_exp= NA', parent=MBsen)
SCzo = Node('SCzo inVoxel=False  n_exp= NA', parent=SCs)
SCsg = Node('SCsg inVoxel=False  n_exp= NA', parent=SCs)
SCop = Node('SCop inVoxel=False  n_exp= NA', parent=SCs)
MEV = Node('MEV inVoxel=True  n_exp= NA', parent=MBsen)
PBG = Node('PBG inVoxel=True  n_exp= NA', parent=MBsen)
SCO = Node('SCO inVoxel=False  n_exp= NA', parent=MBsen)
NB = Node('NB inVoxel=True  n_exp= NA', parent=MBsen)
MBsta = Node('MBsta inVoxel=False  n_exp= NA', parent=MB)
RAmb = Node('RAmb inVoxel=False  n_exp= NA', parent=MBsta)
IF = Node('IF inVoxel=True  n_exp= NA', parent=RAmb)
IPN = Node('IPN inVoxel=True  n_exp= NA', parent=RAmb)
IPC = Node('IPC inVoxel=False  n_exp= NA', parent=IPN)
IPA = Node('IPA inVoxel=False  n_exp= NA', parent=IPN)
IPL = Node('IPL inVoxel=False  n_exp= NA', parent=IPN)
IPRL = Node('IPRL inVoxel=False  n_exp= NA', parent=IPN)
IPR = Node('IPR inVoxel=False  n_exp= NA', parent=IPN)
IPI = Node('IPI inVoxel=False  n_exp= NA', parent=IPN)
IPDM = Node('IPDM inVoxel=False  n_exp= NA', parent=IPN)
IPDL = Node('IPDL inVoxel=False  n_exp= NA', parent=IPN)
RL = Node('RL inVoxel=True  n_exp= NA', parent=RAmb)
CLI = Node('CLI inVoxel=True  n_exp= NA', parent=RAmb)
DR = Node('DR inVoxel=True  n_exp= NA', parent=RAmb)
SNc = Node('SNc inVoxel=True  n_exp= NA', parent=MBsta)
PPN = Node('PPN inVoxel=True  n_exp= NA', parent=MBsta)
HB = Node('HB inVoxel=False  n_exp= NA', parent=BS)
MY = Node('MY inVoxel=False  n_exp= NA', parent=HB)
MY_mot = Node('MY_mot inVoxel=False  n_exp= NA', parent=MY)
IO = Node('IO inVoxel=True  n_exp= NA', parent=MY_mot)
ISN = Node('ISN inVoxel=True  n_exp= NA', parent=MY_mot)
AMB = Node('AMB inVoxel=True  n_exp= NA', parent=MY_mot)
AMBv = Node('AMBv inVoxel=False  n_exp= NA', parent=AMB)
AMBd = Node('AMBd inVoxel=False  n_exp= NA', parent=AMB)
IRN = Node('IRN inVoxel=True  n_exp= NA', parent=MY_mot)
PHY = Node('PHY inVoxel=False  n_exp= NA', parent=MY_mot)
PRP = Node('PRP inVoxel=True  n_exp= NA', parent=PHY)
NR = Node('NR inVoxel=True  n_exp= NA', parent=PHY)
LIN = Node('LIN inVoxel=True  n_exp= NA', parent=MY_mot)
LRN = Node('LRN inVoxel=True  n_exp= NA', parent=MY_mot)
LRNp = Node('LRNp inVoxel=False  n_exp= NA', parent=LRN)
LRNm = Node('LRNm inVoxel=False  n_exp= NA', parent=LRN)
MARN = Node('MARN inVoxel=True  n_exp= NA', parent=MY_mot)
ICB = Node('ICB inVoxel=True  n_exp= NA', parent=MY_mot)
MDRN = Node('MDRN inVoxel=False  n_exp= NA', parent=MY_mot)
MDRNd = Node('MDRNd inVoxel=True  n_exp= NA', parent=MDRN)
MDRNv = Node('MDRNv inVoxel=True  n_exp= NA', parent=MDRN)
VI = Node('VI inVoxel=True  n_exp= NA', parent=MY_mot)
VII = Node('VII inVoxel=True  n_exp= NA', parent=MY_mot)
x = Node('x inVoxel=True  n_exp= NA', parent=MY_mot)
XII = Node('XII inVoxel=True  n_exp= NA', parent=MY_mot)
y = Node('y inVoxel=True  n_exp= NA', parent=MY_mot)
DMX = Node('DMX inVoxel=True  n_exp= NA', parent=MY_mot)
PARN = Node('PARN inVoxel=True  n_exp= NA', parent=MY_mot)
PAS = Node('PAS inVoxel=True  n_exp= NA', parent=MY_mot)
PGRN = Node('PGRN inVoxel=False  n_exp= NA', parent=MY_mot)
PGRNd = Node('PGRNd inVoxel=True  n_exp= NA', parent=PGRN)
PGRNl = Node('PGRNl inVoxel=True  n_exp= NA', parent=PGRN)
GRN = Node('GRN inVoxel=True  n_exp= NA', parent=MY_mot)
PPY = Node('PPY inVoxel=True  n_exp= NA', parent=MY_mot)
ACVII = Node('ACVII inVoxel=True  n_exp= NA', parent=MY_mot)
VNC = Node('VNC inVoxel=False  n_exp= NA', parent=MY_mot)
LAV = Node('LAV inVoxel=True  n_exp= NA', parent=VNC)
SPIV = Node('SPIV inVoxel=True  n_exp= NA', parent=VNC)
MV = Node('MV inVoxel=True  n_exp= NA', parent=VNC)
SUV = Node('SUV inVoxel=True  n_exp= NA', parent=VNC)
MY_sat = Node('MY_sat inVoxel=False  n_exp= NA', parent=MY)
RPA = Node('RPA inVoxel=True  n_exp= NA', parent=MY_sat)
RM = Node('RM inVoxel=True  n_exp= NA', parent=MY_sat)
RO = Node('RO inVoxel=True  n_exp= NA', parent=MY_sat)
MY_sen = Node('MY_sen inVoxel=False  n_exp= NA', parent=MY)
AP = Node('AP inVoxel=True  n_exp= NA', parent=MY_sen)
SPVI = Node('SPVI inVoxel=True  n_exp= NA', parent=MY_sen)
SPVO = Node('SPVO inVoxel=True  n_exp= NA', parent=MY_sen)
CN = Node('CN inVoxel=False  n_exp= NA', parent=MY_sen)
DCO = Node('DCO inVoxel=True  n_exp= NA', parent=CN)
VCO = Node('VCO inVoxel=True  n_exp= NA', parent=CN)
NTB = Node('NTB inVoxel=True  n_exp= NA', parent=MY_sen)
NTS = Node('NTS inVoxel=True  n_exp= NA', parent=MY_sen)
DCN = Node('DCN inVoxel=False  n_exp= NA', parent=MY_sen)
CU = Node('CU inVoxel=True  n_exp= NA', parent=DCN)
GR = Node('GR inVoxel=True  n_exp= NA', parent=DCN)
ECU = Node('ECU inVoxel=True  n_exp= NA', parent=MY_sen)
SPVC = Node('SPVC inVoxel=True  n_exp= NA', parent=MY_sen)
Pa5 = Node('Pa5 inVoxel=False  n_exp= NA', parent=MY_sen)
P = Node('P inVoxel=False  n_exp= NA', parent=HB)
P_mot = Node('P_mot inVoxel=False  n_exp= NA', parent=P)
B = Node('B inVoxel=True  n_exp= NA', parent=P_mot)
SG = Node('SG inVoxel=True  n_exp= NA', parent=P_mot)
SUT = Node('SUT inVoxel=True  n_exp= NA', parent=P_mot)
TRN = Node('TRN inVoxel=True  n_exp= NA', parent=P_mot)
V = Node('V inVoxel=True  n_exp= NA', parent=P_mot)
DTN = Node('DTN inVoxel=True  n_exp= NA', parent=P_mot)
PCG = Node('PCG inVoxel=True  n_exp= NA', parent=P_mot)
PRNc = Node('PRNc inVoxel=True  n_exp= NA', parent=P_mot)
PDTg = Node('PDTg inVoxel=False  n_exp= NA', parent=P_mot)
PG = Node('PG inVoxel=True  n_exp= NA', parent=P_mot)
P5 = Node('P5 inVoxel=False  n_exp= NA', parent=P_mot)
Acs5 = Node('Acs5 inVoxel=False  n_exp= NA', parent=P_mot)
PC5 = Node('PC5 inVoxel=False  n_exp= NA', parent=P_mot)
I5 = Node('I5 inVoxel=False  n_exp= NA', parent=P_mot)
P_sat = Node('P_sat inVoxel=False  n_exp= NA', parent=P)
PRNr = Node('PRNr inVoxel=True  n_exp= NA', parent=P_sat)
LDT = Node('LDT inVoxel=True  n_exp= NA', parent=P_sat)
RPO = Node('RPO inVoxel=True  n_exp= NA', parent=P_sat)
SLC = Node('SLC inVoxel=True  n_exp= NA', parent=P_sat)
SLD = Node('SLD inVoxel=True  n_exp= NA', parent=P_sat)
NI = Node('NI inVoxel=True  n_exp= NA', parent=P_sat)
CS = Node('CS inVoxel=True  n_exp= NA', parent=P_sat)
LC = Node('LC inVoxel=True  n_exp= NA', parent=P_sat)
P_sen = Node('P_sen inVoxel=False  n_exp= NA', parent=P)
PSV = Node('PSV inVoxel=True  n_exp= NA', parent=P_sen)
SOC = Node('SOC inVoxel=True  n_exp= NA', parent=P_sen)
SOCm = Node('SOCm inVoxel=False  n_exp= NA', parent=SOC)
SOCl = Node('SOCl inVoxel=False  n_exp= NA', parent=SOC)
POR = Node('POR inVoxel=False  n_exp= NA', parent=SOC)
NLL = Node('NLL inVoxel=True  n_exp= NA', parent=P_sen)
PB = Node('PB inVoxel=True  n_exp= NA', parent=P_sen)
KF = Node('KF inVoxel=False  n_exp= NA', parent=PB)
IB = Node('IB inVoxel=False  n_exp= NA', parent=BS)
HY = Node('HY inVoxel=False  n_exp= NA', parent=IB)
PVR = Node('PVR inVoxel=False  n_exp= NA', parent=HY)
ADP = Node('ADP inVoxel=True  n_exp= NA', parent=PVR)
PVp = Node('PVp inVoxel=True  n_exp= NA', parent=PVR)
PVpo = Node('PVpo inVoxel=True  n_exp= NA', parent=PVR)
VLPO = Node('VLPO inVoxel=True  n_exp= NA', parent=PVR)
AVP = Node('AVP inVoxel=True  n_exp= NA', parent=PVR)
AVPV = Node('AVPV inVoxel=True  n_exp= NA', parent=PVR)
SCH = Node('SCH inVoxel=True  n_exp= NA', parent=PVR)
SFO = Node('SFO inVoxel=True  n_exp= NA', parent=PVR)
SBPV = Node('SBPV inVoxel=True  n_exp= NA', parent=PVR)
MEPO = Node('MEPO inVoxel=True  n_exp= NA', parent=PVR)
MPO = Node('MPO inVoxel=True  n_exp= NA', parent=PVR)
OV = Node('OV inVoxel=True  n_exp= NA', parent=PVR)
DMH = Node('DMH inVoxel=True  n_exp= NA', parent=PVR)
PD = Node('PD inVoxel=True  n_exp= NA', parent=PVR)
PS = Node('PS inVoxel=True  n_exp= NA', parent=PVR)
VMPO = Node('VMPO inVoxel=False  n_exp= NA', parent=PVR)
PVZ = Node('PVZ inVoxel=False  n_exp= NA', parent=HY)
PVa = Node('PVa inVoxel=True  n_exp= NA', parent=PVZ)
PVH = Node('PVH inVoxel=True  n_exp= NA', parent=PVZ)
PVi = Node('PVi inVoxel=True  n_exp= NA', parent=PVZ)
ARH = Node('ARH inVoxel=True  n_exp= NA', parent=PVZ)
ASO = Node('ASO inVoxel=True  n_exp= NA', parent=PVZ)
SO = Node('SO inVoxel=True  n_exp= NA', parent=PVZ)
LZ = Node('LZ inVoxel=False  n_exp= NA', parent=HY)
RCH = Node('RCH inVoxel=True  n_exp= NA', parent=LZ)
LHA = Node('LHA inVoxel=True  n_exp= NA', parent=LZ)
LPO = Node('LPO inVoxel=True  n_exp= NA', parent=LZ)
PST = Node('PST inVoxel=True  n_exp= NA', parent=LZ)
PSTN = Node('PSTN inVoxel=True  n_exp= NA', parent=LZ)
STN = Node('STN inVoxel=True  n_exp= NA', parent=LZ)
TU = Node('TU inVoxel=True  n_exp= NA', parent=LZ)
ZI = Node('ZI inVoxel=True  n_exp= NA', parent=LZ)
FF = Node('FF inVoxel=False  n_exp= NA', parent=ZI)
PeF = Node('PeF inVoxel=False  n_exp= NA', parent=LZ)
MEZ = Node('MEZ inVoxel=False  n_exp= NA', parent=HY)
PVHd = Node('PVHd inVoxel=True  n_exp= NA', parent=MEZ)
AHN = Node('AHN inVoxel=True  n_exp= NA', parent=MEZ)
MBO = Node('MBO inVoxel=False  n_exp= NA', parent=MEZ)
LM = Node('LM inVoxel=True  n_exp= NA', parent=MBO)
MM = Node('MM inVoxel=True  n_exp= NA', parent=MBO)
MMme = Node('MMme inVoxel=False  n_exp= NA', parent=MM)
MMm = Node('MMm inVoxel=False  n_exp= NA', parent=MM)
MMd = Node('MMd inVoxel=False  n_exp= NA', parent=MM)
MMl = Node('MMl inVoxel=False  n_exp= NA', parent=MM)
MMp = Node('MMp inVoxel=False  n_exp= NA', parent=MM)
TM = Node('TM inVoxel=False  n_exp= NA', parent=MBO)
TMv = Node('TMv inVoxel=True  n_exp= NA', parent=TM)
TMd = Node('TMd inVoxel=True  n_exp= NA', parent=TM)
SUM = Node('SUM inVoxel=True  n_exp= NA', parent=MBO)
MPN = Node('MPN inVoxel=True  n_exp= NA', parent=MEZ)
VMH = Node('VMH inVoxel=True  n_exp= NA', parent=MEZ)
PH = Node('PH inVoxel=True  n_exp= NA', parent=MEZ)
PMd = Node('PMd inVoxel=True  n_exp= NA', parent=MEZ)
PMv = Node('PMv inVoxel=True  n_exp= NA', parent=MEZ)
ME = Node('ME inVoxel=False  n_exp= NA', parent=HY)
TH = Node('TH inVoxel=False  n_exp= NA', parent=IB)
DORpm = Node('DORpm inVoxel=False  n_exp= NA', parent=TH)
ILM = Node('ILM inVoxel=False  n_exp= NA', parent=DORpm)
RH = Node('RH inVoxel=True  n_exp= NA', parent=ILM)
CL = Node('CL inVoxel=True  n_exp= NA', parent=ILM)
CM = Node('CM inVoxel=True  n_exp= NA', parent=ILM)
PCN = Node('PCN inVoxel=True  n_exp= NA', parent=ILM)
PF = Node('PF inVoxel=True  n_exp= NA', parent=ILM)
PIL = Node('PIL inVoxel=False  n_exp= NA', parent=ILM)
LAT = Node('LAT inVoxel=False  n_exp= NA', parent=DORpm)
LP = Node('LP inVoxel=True  n_exp= NA', parent=LAT)
SGN = Node('SGN inVoxel=True  n_exp= NA', parent=LAT)
PO = Node('PO inVoxel=True  n_exp= NA', parent=LAT)
POL = Node('POL inVoxel=True  n_exp= NA', parent=LAT)
Eth = Node('Eth inVoxel=False  n_exp= NA', parent=LAT)
ATN = Node('ATN inVoxel=False  n_exp= NA', parent=DORpm)
AD = Node('AD inVoxel=True  n_exp= NA', parent=ATN)
AM = Node('AM inVoxel=True  n_exp= NA', parent=ATN)
AMd = Node('AMd inVoxel=False  n_exp= NA', parent=AM)
AMv = Node('AMv inVoxel=False  n_exp= NA', parent=AM)
LD = Node('LD inVoxel=True  n_exp= NA', parent=ATN)
AV = Node('AV inVoxel=True  n_exp= NA', parent=ATN)
IAD = Node('IAD inVoxel=True  n_exp= NA', parent=ATN)
IAM = Node('IAM inVoxel=True  n_exp= NA', parent=ATN)
RT = Node('RT inVoxel=True  n_exp= NA', parent=DORpm)
MED = Node('MED inVoxel=False  n_exp= NA', parent=DORpm)
IMD = Node('IMD inVoxel=True  n_exp= NA', parent=MED)
MD = Node('MD inVoxel=True  n_exp= NA', parent=MED)
SMT = Node('SMT inVoxel=True  n_exp= NA', parent=MED)
PR = Node('PR inVoxel=True  n_exp= NA', parent=MED)
MTN = Node('MTN inVoxel=False  n_exp= NA', parent=DORpm)
PT = Node('PT inVoxel=True  n_exp= NA', parent=MTN)
PVT = Node('PVT inVoxel=True  n_exp= NA', parent=MTN)
RE = Node('RE inVoxel=True  n_exp= NA', parent=MTN)
Xi = Node('Xi inVoxel=False  n_exp= NA', parent=MTN)
EPI = Node('EPI inVoxel=False  n_exp= NA', parent=DORpm)
LH = Node('LH inVoxel=True  n_exp= NA', parent=EPI)
MH = Node('MH inVoxel=True  n_exp= NA', parent=EPI)
GENv = Node('GENv inVoxel=False  n_exp= NA', parent=DORpm)
IGL = Node('IGL inVoxel=True  n_exp= NA', parent=GENv)
LGv = Node('LGv inVoxel=True  n_exp= NA', parent=GENv)
SubG = Node('SubG inVoxel=True  n_exp= NA', parent=GENv)
IntG = Node('IntG inVoxel=False  n_exp= NA', parent=GENv)
DORsm = Node('DORsm inVoxel=False  n_exp= NA', parent=TH)
SPF = Node('SPF inVoxel=False  n_exp= NA', parent=DORsm)
SPFm = Node('SPFm inVoxel=True  n_exp= NA', parent=SPF)
SPFp = Node('SPFp inVoxel=True  n_exp= NA', parent=SPF)
SPA = Node('SPA inVoxel=True  n_exp= NA', parent=DORsm)
VENT = Node('VENT inVoxel=False  n_exp= NA', parent=DORsm)
VM = Node('VM inVoxel=True  n_exp= NA', parent=VENT)
PoT = Node('PoT inVoxel=False  n_exp= NA', parent=VENT)
VAL = Node('VAL inVoxel=True  n_exp= NA', parent=VENT)
VP = Node('VP inVoxel=False  n_exp= NA', parent=VENT)
VPL = Node('VPL inVoxel=True  n_exp= NA', parent=VP)
VPLpc = Node('VPLpc inVoxel=True  n_exp= NA', parent=VP)
VPM = Node('VPM inVoxel=True  n_exp= NA', parent=VP)
VPMpc = Node('VPMpc inVoxel=True  n_exp= NA', parent=VP)
GENd = Node('GENd inVoxel=False  n_exp= NA', parent=DORsm)
LGd = Node('LGd inVoxel=True  n_exp= NA', parent=GENd)
LGd_sh = Node('LGd_sh inVoxel=False  n_exp= NA', parent=LGd)
LGd_co = Node('LGd_co inVoxel=False  n_exp= NA', parent=LGd)
LGd_ip = Node('LGd_ip inVoxel=False  n_exp= NA', parent=LGd)
MG = Node('MG inVoxel=True  n_exp= NA', parent=GENd)
MGd = Node('MGd inVoxel=False  n_exp= NA', parent=MG)
MGv = Node('MGv inVoxel=False  n_exp= NA', parent=MG)
MGm = Node('MGm inVoxel=False  n_exp= NA', parent=MG)
PP = Node('PP inVoxel=True  n_exp= NA', parent=DORsm)

In [42]:
for pre, fill, node in RenderTree(BS):
    print   pre,node.name

 BS inVoxel=False
├──  MB inVoxel=False  n_exp= NA
│   ├──  MBmot inVoxel=False  n_exp= NA
│   │   ├──  III inVoxel=True  n_exp= NA
│   │   ├──  MT inVoxel=False  n_exp= NA
│   │   ├──  LT inVoxel=True  n_exp= NA
│   │   ├──  DT inVoxel=False  n_exp= NA
│   │   ├──  IV inVoxel=True  n_exp= NA
│   │   ├──  MRN inVoxel=True  n_exp= NA
│   │   ├──  PN inVoxel=False  n_exp= NA
│   │   ├──  RN inVoxel=True  n_exp= NA
│   │   ├──  AT inVoxel=True  n_exp= NA
│   │   ├──  RR inVoxel=True  n_exp= NA
│   │   ├──  SCm inVoxel=True  n_exp= NA
│   │   │   ├──  SCig inVoxel=False  n_exp= NA
│   │   │   │   ├──  SCig_a inVoxel=False  n_exp= NA
│   │   │   │   ├──  SCig_b inVoxel=False  n_exp= NA
│   │   │   │   └──  SCig_c inVoxel=False  n_exp= NA
│   │   │   ├──  SCiw inVoxel=False  n_exp= NA
│   │   │   ├──  SCdg inVoxel=False  n_exp= NA
│   │   │   └──  SCdw inVoxel=False  n_exp= NA
│   │   ├──  CUN inVoxel=True  n_exp= NA
│   │   ├──  VTA inVoxel=True  n_exp= NA
│   │   ├──  VTN inVoxel=True  n_e

In [55]:
print_code_for_tree('CTX')

CTXpl = Node('CTXpl inVoxel=False  n_exp= NA', parent=CTX)
Isocortex = Node('Isocortex inVoxel=False  n_exp= NA', parent=CTXpl)
PTLp = Node('PTLp inVoxel=False  n_exp= NA', parent=Isocortex)
VISa = Node('VISa inVoxel=True  n_exp= NA', parent=PTLp)
VISa1 = Node('VISa1 inVoxel=False  n_exp= NA', parent=VISa)
VISa2and3 = Node('VISa2and3 inVoxel=False  n_exp= NA', parent=VISa)
VISa4 = Node('VISa4 inVoxel=False  n_exp= NA', parent=VISa)
VISa5 = Node('VISa5 inVoxel=False  n_exp= NA', parent=VISa)
VISa6a = Node('VISa6a inVoxel=False  n_exp= NA', parent=VISa)
VISa6b = Node('VISa6b inVoxel=False  n_exp= NA', parent=VISa)
VISrl = Node('VISrl inVoxel=True  n_exp= NA', parent=PTLp)
VISrl1 = Node('VISrl1 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl2and3 = Node('VISrl2and3 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl4 = Node('VISrl4 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl5 = Node('VISrl5 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6a = Node('VISrl6a inVoxel=False  n_exp= NA', parent=VIS

In [56]:
CTXpl = Node('CTXpl inVoxel=False  n_exp= NA', parent=CTX)
Isocortex = Node('Isocortex inVoxel=False  n_exp= NA', parent=CTXpl)
PTLp = Node('PTLp inVoxel=False  n_exp= NA', parent=Isocortex)
VISa = Node('VISa inVoxel=True  n_exp= NA', parent=PTLp)
VISa1 = Node('VISa1 inVoxel=False  n_exp= NA', parent=VISa)
VISa2and3 = Node('VISa2and3 inVoxel=False  n_exp= NA', parent=VISa)
VISa4 = Node('VISa4 inVoxel=False  n_exp= NA', parent=VISa)
VISa5 = Node('VISa5 inVoxel=False  n_exp= NA', parent=VISa)
VISa6a = Node('VISa6a inVoxel=False  n_exp= NA', parent=VISa)
VISa6b = Node('VISa6b inVoxel=False  n_exp= NA', parent=VISa)
VISrl = Node('VISrl inVoxel=True  n_exp= NA', parent=PTLp)
VISrl1 = Node('VISrl1 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl2and3 = Node('VISrl2and3 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl4 = Node('VISrl4 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl5 = Node('VISrl5 inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6a = Node('VISrl6a inVoxel=False  n_exp= NA', parent=VISrl)
VISrl6b = Node('VISrl6b inVoxel=False  n_exp= NA', parent=VISrl)
ACA = Node('ACA inVoxel=False  n_exp= NA', parent=Isocortex)
ACAd = Node('ACAd inVoxel=True  n_exp= NA', parent=ACA)
ACAd2and3 = Node('ACAd2and3 inVoxel=False  n_exp= NA', parent=ACAd)
ACAd6a = Node('ACAd6a inVoxel=False  n_exp= NA', parent=ACAd)
ACAd6b = Node('ACAd6b inVoxel=False  n_exp= NA', parent=ACAd)
ACAd1 = Node('ACAd1 inVoxel=False  n_exp= NA', parent=ACAd)
ACAd5 = Node('ACAd5 inVoxel=False  n_exp= NA', parent=ACAd)
ACAv = Node('ACAv inVoxel=True  n_exp= NA', parent=ACA)
ACAv2and3 = Node('ACAv2and3 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv1 = Node('ACAv1 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv5 = Node('ACAv5 inVoxel=False  n_exp= NA', parent=ACAv)
ACAv6a = Node('ACAv6a inVoxel=False  n_exp= NA', parent=ACAv)
ACAv6b = Node('ACAv6b inVoxel=False  n_exp= NA', parent=ACAv)
ILA = Node('ILA inVoxel=True  n_exp= NA', parent=Isocortex)
ILA2and3 = Node('ILA2and3 inVoxel=False  n_exp= NA', parent=ILA)
ILA1 = Node('ILA1 inVoxel=False  n_exp= NA', parent=ILA)
ILA5 = Node('ILA5 inVoxel=False  n_exp= NA', parent=ILA)
ILA6a = Node('ILA6a inVoxel=False  n_exp= NA', parent=ILA)
ILA6b = Node('ILA6b inVoxel=False  n_exp= NA', parent=ILA)
AI = Node('AI inVoxel=False  n_exp= NA', parent=Isocortex)
AId = Node('AId inVoxel=True  n_exp= NA', parent=AI)
AId2and3 = Node('AId2and3 inVoxel=False  n_exp= NA', parent=AId)
AId6a = Node('AId6a inVoxel=False  n_exp= NA', parent=AId)
AId6b = Node('AId6b inVoxel=False  n_exp= NA', parent=AId)
AId1 = Node('AId1 inVoxel=False  n_exp= NA', parent=AId)
AId5 = Node('AId5 inVoxel=False  n_exp= NA', parent=AId)
AIp = Node('AIp inVoxel=True  n_exp= NA', parent=AI)
AIp1 = Node('AIp1 inVoxel=False  n_exp= NA', parent=AIp)
AIp2and3 = Node('AIp2and3 inVoxel=False  n_exp= NA', parent=AIp)
AIp6a = Node('AIp6a inVoxel=False  n_exp= NA', parent=AIp)
AIp5 = Node('AIp5 inVoxel=False  n_exp= NA', parent=AIp)
AIp6b = Node('AIp6b inVoxel=False  n_exp= NA', parent=AIp)
AIv = Node('AIv inVoxel=True  n_exp= NA', parent=AI)
AIv6a = Node('AIv6a inVoxel=False  n_exp= NA', parent=AIv)
AIv2and3 = Node('AIv2and3 inVoxel=False  n_exp= NA', parent=AIv)
AIv6b = Node('AIv6b inVoxel=False  n_exp= NA', parent=AIv)
AIv1 = Node('AIv1 inVoxel=False  n_exp= NA', parent=AIv)
AIv5 = Node('AIv5 inVoxel=False  n_exp= NA', parent=AIv)
FRP = Node('FRP inVoxel=True  n_exp= NA', parent=Isocortex)
FRP1 = Node('FRP1 inVoxel=False  n_exp= NA', parent=FRP)
FRP2and3 = Node('FRP2and3 inVoxel=False  n_exp= NA', parent=FRP)
FRP5 = Node('FRP5 inVoxel=False  n_exp= NA', parent=FRP)
FRP6a = Node('FRP6a inVoxel=False  n_exp= NA', parent=FRP)
FRP6b = Node('FRP6b inVoxel=False  n_exp= NA', parent=FRP)
AUD = Node('AUD inVoxel=False  n_exp= NA', parent=Isocortex)
AUDp = Node('AUDp inVoxel=True  n_exp= NA', parent=AUD)
AUDp2and3 = Node('AUDp2and3 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp1 = Node('AUDp1 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp4 = Node('AUDp4 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp5 = Node('AUDp5 inVoxel=False  n_exp= NA', parent=AUDp)
AUDp6a = Node('AUDp6a inVoxel=False  n_exp= NA', parent=AUDp)
AUDp6b = Node('AUDp6b inVoxel=False  n_exp= NA', parent=AUDp)
AUDd = Node('AUDd inVoxel=True  n_exp= NA', parent=AUD)
AUDd6a = Node('AUDd6a inVoxel=False  n_exp= NA', parent=AUDd)
AUDd6b = Node('AUDd6b inVoxel=False  n_exp= NA', parent=AUDd)
AUDd5 = Node('AUDd5 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd1 = Node('AUDd1 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd2and3 = Node('AUDd2and3 inVoxel=False  n_exp= NA', parent=AUDd)
AUDd4 = Node('AUDd4 inVoxel=False  n_exp= NA', parent=AUDd)
AUDv = Node('AUDv inVoxel=True  n_exp= NA', parent=AUD)
AUDv6a = Node('AUDv6a inVoxel=False  n_exp= NA', parent=AUDv)
AUDv6b = Node('AUDv6b inVoxel=False  n_exp= NA', parent=AUDv)
AUDv2and3 = Node('AUDv2and3 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv1 = Node('AUDv1 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv4 = Node('AUDv4 inVoxel=False  n_exp= NA', parent=AUDv)
AUDv5 = Node('AUDv5 inVoxel=False  n_exp= NA', parent=AUDv)
AUDpo = Node('AUDpo inVoxel=True  n_exp= NA', parent=AUD)
AUDpo6a = Node('AUDpo6a inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo6b = Node('AUDpo6b inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo2and3 = Node('AUDpo2and3 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo1 = Node('AUDpo1 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo4 = Node('AUDpo4 inVoxel=False  n_exp= NA', parent=AUDpo)
AUDpo5 = Node('AUDpo5 inVoxel=False  n_exp= NA', parent=AUDpo)
RSP = Node('RSP inVoxel=False  n_exp= NA', parent=Isocortex)
RSPd = Node('RSPd inVoxel=True  n_exp= NA', parent=RSP)
RSPd6a = Node('RSPd6a inVoxel=False  n_exp= NA', parent=RSPd)
RSPd6b = Node('RSPd6b inVoxel=False  n_exp= NA', parent=RSPd)
RSPd2and3 = Node('RSPd2and3 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd1 = Node('RSPd1 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd4 = Node('RSPd4 inVoxel=False  n_exp= NA', parent=RSPd)
RSPd5 = Node('RSPd5 inVoxel=False  n_exp= NA', parent=RSPd)
RSPv = Node('RSPv inVoxel=True  n_exp= NA', parent=RSP)
RSPv2and3 = Node('RSPv2and3 inVoxel=False  n_exp= NA', parent=RSPv)
RSPv1 = Node('RSPv1 inVoxel=False  n_exp= NA', parent=RSPv)
RSPv6a = Node('RSPv6a inVoxel=False  n_exp= NA', parent=RSPv)
RSPv6b = Node('RSPv6b inVoxel=False  n_exp= NA', parent=RSPv)
RSPv5 = Node('RSPv5 inVoxel=False  n_exp= NA', parent=RSPv)
RSPagl = Node('RSPagl inVoxel=True  n_exp= NA', parent=RSP)
RSPagl6b = Node('RSPagl6b inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl1 = Node('RSPagl1 inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl5 = Node('RSPagl5 inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl6a = Node('RSPagl6a inVoxel=False  n_exp= NA', parent=RSPagl)
RSPagl2and3 = Node('RSPagl2and3 inVoxel=False  n_exp= NA', parent=RSPagl)
MO = Node('MO inVoxel=False  n_exp= NA', parent=Isocortex)
MOp = Node('MOp inVoxel=True  n_exp= NA', parent=MO)
MOp1 = Node('MOp1 inVoxel=False  n_exp= NA', parent=MOp)
MOp5 = Node('MOp5 inVoxel=False  n_exp= NA', parent=MOp)
MOp6a = Node('MOp6a inVoxel=False  n_exp= NA', parent=MOp)
MOp6b = Node('MOp6b inVoxel=False  n_exp= NA', parent=MOp)
MOp2and3 = Node('MOp2and3 inVoxel=False  n_exp= NA', parent=MOp)
MOs = Node('MOs inVoxel=True  n_exp= NA', parent=MO)
MOs1 = Node('MOs1 inVoxel=False  n_exp= NA', parent=MOs)
MOs5 = Node('MOs5 inVoxel=False  n_exp= NA', parent=MOs)
MOs2and3 = Node('MOs2and3 inVoxel=False  n_exp= NA', parent=MOs)
MOs6a = Node('MOs6a inVoxel=False  n_exp= NA', parent=MOs)
MOs6b = Node('MOs6b inVoxel=False  n_exp= NA', parent=MOs)
TEa = Node('TEa inVoxel=True  n_exp= NA', parent=Isocortex)
TEa1 = Node('TEa1 inVoxel=False  n_exp= NA', parent=TEa)
TEa4 = Node('TEa4 inVoxel=False  n_exp= NA', parent=TEa)
TEa5 = Node('TEa5 inVoxel=False  n_exp= NA', parent=TEa)
TEa6a = Node('TEa6a inVoxel=False  n_exp= NA', parent=TEa)
TEa6b = Node('TEa6b inVoxel=False  n_exp= NA', parent=TEa)
TEa2and3 = Node('TEa2and3 inVoxel=False  n_exp= NA', parent=TEa)
VIS = Node('VIS inVoxel=False  n_exp= NA', parent=Isocortex)
VISp = Node('VISp inVoxel=True  n_exp= NA', parent=VIS)
VISp6a = Node('VISp6a inVoxel=False  n_exp= NA', parent=VISp)
VISp6b = Node('VISp6b inVoxel=False  n_exp= NA', parent=VISp)
VISp1 = Node('VISp1 inVoxel=False  n_exp= NA', parent=VISp)
VISp4 = Node('VISp4 inVoxel=False  n_exp= NA', parent=VISp)
VISp5 = Node('VISp5 inVoxel=False  n_exp= NA', parent=VISp)
VISp2and3 = Node('VISp2and3 inVoxel=False  n_exp= NA', parent=VISp)
VISam = Node('VISam inVoxel=True  n_exp= NA', parent=VIS)
VISam1 = Node('VISam1 inVoxel=False  n_exp= NA', parent=VISam)
VISam4 = Node('VISam4 inVoxel=False  n_exp= NA', parent=VISam)
VISam5 = Node('VISam5 inVoxel=False  n_exp= NA', parent=VISam)
VISam6b = Node('VISam6b inVoxel=False  n_exp= NA', parent=VISam)
VISam6a = Node('VISam6a inVoxel=False  n_exp= NA', parent=VISam)
VISam2and3 = Node('VISam2and3 inVoxel=False  n_exp= NA', parent=VISam)
VISal = Node('VISal inVoxel=True  n_exp= NA', parent=VIS)
VISal5 = Node('VISal5 inVoxel=False  n_exp= NA', parent=VISal)
VISal6a = Node('VISal6a inVoxel=False  n_exp= NA', parent=VISal)
VISal6b = Node('VISal6b inVoxel=False  n_exp= NA', parent=VISal)
VISal2and3 = Node('VISal2and3 inVoxel=False  n_exp= NA', parent=VISal)
VISal1 = Node('VISal1 inVoxel=False  n_exp= NA', parent=VISal)
VISal4 = Node('VISal4 inVoxel=False  n_exp= NA', parent=VISal)
VISli = Node('VISli inVoxel=True  n_exp= NA', parent=VIS)
VISli2and3 = Node('VISli2and3 inVoxel=False  n_exp= NA', parent=VISli)
VISli4 = Node('VISli4 inVoxel=False  n_exp= NA', parent=VISli)
VISli5 = Node('VISli5 inVoxel=False  n_exp= NA', parent=VISli)
VISli1 = Node('VISli1 inVoxel=False  n_exp= NA', parent=VISli)
VISli6a = Node('VISli6a inVoxel=False  n_exp= NA', parent=VISli)
VISli6b = Node('VISli6b inVoxel=False  n_exp= NA', parent=VISli)
VISl = Node('VISl inVoxel=True  n_exp= NA', parent=VIS)
VISl6a = Node('VISl6a inVoxel=False  n_exp= NA', parent=VISl)
VISl6b = Node('VISl6b inVoxel=False  n_exp= NA', parent=VISl)
VISl1 = Node('VISl1 inVoxel=False  n_exp= NA', parent=VISl)
VISl4 = Node('VISl4 inVoxel=False  n_exp= NA', parent=VISl)
VISl5 = Node('VISl5 inVoxel=False  n_exp= NA', parent=VISl)
VISl2and3 = Node('VISl2and3 inVoxel=False  n_exp= NA', parent=VISl)
VISpl = Node('VISpl inVoxel=True  n_exp= NA', parent=VIS)
VISpl2and3 = Node('VISpl2and3 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl6a = Node('VISpl6a inVoxel=False  n_exp= NA', parent=VISpl)
VISpl6b = Node('VISpl6b inVoxel=False  n_exp= NA', parent=VISpl)
VISpl1 = Node('VISpl1 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl4 = Node('VISpl4 inVoxel=False  n_exp= NA', parent=VISpl)
VISpl5 = Node('VISpl5 inVoxel=False  n_exp= NA', parent=VISpl)
VISpm = Node('VISpm inVoxel=True  n_exp= NA', parent=VIS)
VISpm2and3 = Node('VISpm2and3 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm6a = Node('VISpm6a inVoxel=False  n_exp= NA', parent=VISpm)
VISpm6b = Node('VISpm6b inVoxel=False  n_exp= NA', parent=VISpm)
VISpm5 = Node('VISpm5 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm4 = Node('VISpm4 inVoxel=False  n_exp= NA', parent=VISpm)
VISpm1 = Node('VISpm1 inVoxel=False  n_exp= NA', parent=VISpm)
VISpor = Node('VISpor inVoxel=True  n_exp= NA', parent=VIS)
VISpor1 = Node('VISpor1 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor2and3 = Node('VISpor2and3 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor4 = Node('VISpor4 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor5 = Node('VISpor5 inVoxel=False  n_exp= NA', parent=VISpor)
VISpor6a = Node('VISpor6a inVoxel=False  n_exp= NA', parent=VISpor)
VISpor6b = Node('VISpor6b inVoxel=False  n_exp= NA', parent=VISpor)
VISC = Node('VISC inVoxel=True  n_exp= NA', parent=Isocortex)
VISC6b = Node('VISC6b inVoxel=False  n_exp= NA', parent=VISC)
VISC6a = Node('VISC6a inVoxel=False  n_exp= NA', parent=VISC)
VISC1 = Node('VISC1 inVoxel=False  n_exp= NA', parent=VISC)
VISC4 = Node('VISC4 inVoxel=False  n_exp= NA', parent=VISC)
VISC5 = Node('VISC5 inVoxel=False  n_exp= NA', parent=VISC)
VISC2and3 = Node('VISC2and3 inVoxel=False  n_exp= NA', parent=VISC)
ORB = Node('ORB inVoxel=False  n_exp= NA', parent=Isocortex)
ORBl = Node('ORBl inVoxel=True  n_exp= NA', parent=ORB)
ORBl2and3 = Node('ORBl2and3 inVoxel=False  n_exp= NA', parent=ORBl)
ORBl6a = Node('ORBl6a inVoxel=False  n_exp= NA', parent=ORBl)
ORBl1 = Node('ORBl1 inVoxel=False  n_exp= NA', parent=ORBl)
ORBl6b = Node('ORBl6b inVoxel=False  n_exp= NA', parent=ORBl)
ORBl5 = Node('ORBl5 inVoxel=False  n_exp= NA', parent=ORBl)
ORBm = Node('ORBm inVoxel=True  n_exp= NA', parent=ORB)
ORBm1 = Node('ORBm1 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm2and3 = Node('ORBm2and3 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm5 = Node('ORBm5 inVoxel=False  n_exp= NA', parent=ORBm)
ORBm6a = Node('ORBm6a inVoxel=False  n_exp= NA', parent=ORBm)
ORBm6b = Node('ORBm6b inVoxel=False  n_exp= NA', parent=ORBm)
ORBvl = Node('ORBvl inVoxel=True  n_exp= NA', parent=ORB)
ORBvl2and3 = Node('ORBvl2and3 inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl6a = Node('ORBvl6a inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl6b = Node('ORBvl6b inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl1 = Node('ORBvl1 inVoxel=False  n_exp= NA', parent=ORBvl)
ORBvl5 = Node('ORBvl5 inVoxel=False  n_exp= NA', parent=ORBvl)
ECT = Node('ECT inVoxel=True  n_exp= NA', parent=Isocortex)
ECT2and3 = Node('ECT2and3 inVoxel=False  n_exp= NA', parent=ECT)
ECT1 = Node('ECT1 inVoxel=False  n_exp= NA', parent=ECT)
ECT6a = Node('ECT6a inVoxel=False  n_exp= NA', parent=ECT)
ECT5 = Node('ECT5 inVoxel=False  n_exp= NA', parent=ECT)
ECT6b = Node('ECT6b inVoxel=False  n_exp= NA', parent=ECT)
PERI = Node('PERI inVoxel=True  n_exp= NA', parent=Isocortex)
PERI6a = Node('PERI6a inVoxel=False  n_exp= NA', parent=PERI)
PERI6b = Node('PERI6b inVoxel=False  n_exp= NA', parent=PERI)
PERI1 = Node('PERI1 inVoxel=False  n_exp= NA', parent=PERI)
PERI5 = Node('PERI5 inVoxel=False  n_exp= NA', parent=PERI)
PERI2and3 = Node('PERI2and3 inVoxel=False  n_exp= NA', parent=PERI)
PL = Node('PL inVoxel=True  n_exp= NA', parent=Isocortex)
PL6a = Node('PL6a inVoxel=False  n_exp= NA', parent=PL)
PL6b = Node('PL6b inVoxel=False  n_exp= NA', parent=PL)
PL1 = Node('PL1 inVoxel=False  n_exp= NA', parent=PL)
PL2and3 = Node('PL2and3 inVoxel=False  n_exp= NA', parent=PL)
PL5 = Node('PL5 inVoxel=False  n_exp= NA', parent=PL)
GU = Node('GU inVoxel=True  n_exp= NA', parent=Isocortex)
GU1 = Node('GU1 inVoxel=False  n_exp= NA', parent=GU)
GU4 = Node('GU4 inVoxel=False  n_exp= NA', parent=GU)
GU2and3 = Node('GU2and3 inVoxel=False  n_exp= NA', parent=GU)
GU5 = Node('GU5 inVoxel=False  n_exp= NA', parent=GU)
GU6a = Node('GU6a inVoxel=False  n_exp= NA', parent=GU)
GU6b = Node('GU6b inVoxel=False  n_exp= NA', parent=GU)
SS = Node('SS inVoxel=False  n_exp= NA', parent=Isocortex)
SSp = Node('SSp inVoxel=False  n_exp= NA', parent=SS)
SSp_bfd = Node('SSp_bfd inVoxel=True  n_exp= NA', parent=SSp)
SSp_bfd2and3 = Node('SSp_bfd2and3 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd1 = Node('SSp_bfd1 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd6a = Node('SSp_bfd6a inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd4 = Node('SSp_bfd4 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd6b = Node('SSp_bfd6b inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_bfd5 = Node('SSp_bfd5 inVoxel=False  n_exp= NA', parent=SSp_bfd)
SSp_ll = Node('SSp_ll inVoxel=True  n_exp= NA', parent=SSp)
SSp_ll2and3 = Node('SSp_ll2and3 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll6a = Node('SSp_ll6a inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll6b = Node('SSp_ll6b inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll1 = Node('SSp_ll1 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll4 = Node('SSp_ll4 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_ll5 = Node('SSp_ll5 inVoxel=False  n_exp= NA', parent=SSp_ll)
SSp_m = Node('SSp_m inVoxel=True  n_exp= NA', parent=SSp)
SSp_m6b = Node('SSp_m6b inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m2and3 = Node('SSp_m2and3 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m1 = Node('SSp_m1 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m4 = Node('SSp_m4 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m5 = Node('SSp_m5 inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_m6a = Node('SSp_m6a inVoxel=False  n_exp= NA', parent=SSp_m)
SSp_n = Node('SSp_n inVoxel=True  n_exp= NA', parent=SSp)
SSp_n1 = Node('SSp_n1 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n4 = Node('SSp_n4 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n5 = Node('SSp_n5 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n2and3 = Node('SSp_n2and3 inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n6a = Node('SSp_n6a inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_n6b = Node('SSp_n6b inVoxel=False  n_exp= NA', parent=SSp_n)
SSp_tr = Node('SSp_tr inVoxel=True  n_exp= NA', parent=SSp)
SSp_tr6a = Node('SSp_tr6a inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr6b = Node('SSp_tr6b inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr2and3 = Node('SSp_tr2and3 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr1 = Node('SSp_tr1 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr4 = Node('SSp_tr4 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_tr5 = Node('SSp_tr5 inVoxel=False  n_exp= NA', parent=SSp_tr)
SSp_ul = Node('SSp_ul inVoxel=True  n_exp= NA', parent=SSp)
SSp_ul1 = Node('SSp_ul1 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul4 = Node('SSp_ul4 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul5 = Node('SSp_ul5 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul2and3 = Node('SSp_ul2and3 inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul6b = Node('SSp_ul6b inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_ul6a = Node('SSp_ul6a inVoxel=False  n_exp= NA', parent=SSp_ul)
SSp_un = Node('SSp_un inVoxel=True  n_exp= NA', parent=SSp)
SSp_un1 = Node('SSp_un1 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un4 = Node('SSp_un4 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un5 = Node('SSp_un5 inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un6a = Node('SSp_un6a inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un6b = Node('SSp_un6b inVoxel=False  n_exp= NA', parent=SSp_un)
SSp_un2and3 = Node('SSp_un2and3 inVoxel=False  n_exp= NA', parent=SSp_un)
SSs = Node('SSs inVoxel=True  n_exp= NA', parent=SS)
SSs2and3 = Node('SSs2and3 inVoxel=False  n_exp= NA', parent=SSs)
SSs6a = Node('SSs6a inVoxel=False  n_exp= NA', parent=SSs)
SSs1 = Node('SSs1 inVoxel=False  n_exp= NA', parent=SSs)
SSs6b = Node('SSs6b inVoxel=False  n_exp= NA', parent=SSs)
SSs4 = Node('SSs4 inVoxel=False  n_exp= NA', parent=SSs)
SSs5 = Node('SSs5 inVoxel=False  n_exp= NA', parent=SSs)
OLF = Node('OLF inVoxel=False  n_exp= NA', parent=CTXpl)
AOB = Node('AOB inVoxel=True  n_exp= NA', parent=OLF)
AOBgl = Node('AOBgl inVoxel=False  n_exp= NA', parent=AOB)
AOBgr = Node('AOBgr inVoxel=False  n_exp= NA', parent=AOB)
AOBmi = Node('AOBmi inVoxel=False  n_exp= NA', parent=AOB)
AON = Node('AON inVoxel=True  n_exp= NA', parent=OLF)
MOB = Node('MOB inVoxel=True  n_exp= NA', parent=OLF)
TR = Node('TR inVoxel=True  n_exp= NA', parent=OLF)
TT = Node('TT inVoxel=True  n_exp= NA', parent=OLF)
TTd = Node('TTd inVoxel=False  n_exp= NA', parent=TT)
TTv = Node('TTv inVoxel=False  n_exp= NA', parent=TT)
NLOT = Node('NLOT inVoxel=True  n_exp= NA', parent=OLF)
NLOT1 = Node('NLOT1 inVoxel=False  n_exp= NA', parent=NLOT)
NLOT2 = Node('NLOT2 inVoxel=False  n_exp= NA', parent=NLOT)
NLOT3 = Node('NLOT3 inVoxel=False  n_exp= NA', parent=NLOT)
COA = Node('COA inVoxel=False  n_exp= NA', parent=OLF)
COAa = Node('COAa inVoxel=True  n_exp= NA', parent=COA)
COAp = Node('COAp inVoxel=True  n_exp= NA', parent=COA)
COApl = Node('COApl inVoxel=False  n_exp= NA', parent=COAp)
COApm = Node('COApm inVoxel=False  n_exp= NA', parent=COAp)
PAA = Node('PAA inVoxel=True  n_exp= NA', parent=OLF)
DP = Node('DP inVoxel=True  n_exp= NA', parent=OLF)
PIR = Node('PIR inVoxel=True  n_exp= NA', parent=OLF)
HPF = Node('HPF inVoxel=False  n_exp= NA', parent=CTXpl)
RHP = Node('RHP inVoxel=False  n_exp= NA', parent=HPF)
SUB = Node('SUB inVoxel=True  n_exp= NA', parent=RHP)
APr = Node('APr inVoxel=False  n_exp= NA', parent=RHP)
PAR = Node('PAR inVoxel=True  n_exp= NA', parent=RHP)
ENT = Node('ENT inVoxel=False  n_exp= NA', parent=RHP)
ENTl = Node('ENTl inVoxel=True  n_exp= NA', parent=ENT)
ENTl2 = Node('ENTl2 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl6a = Node('ENTl6a inVoxel=False  n_exp= NA', parent=ENTl)
ENTl3 = Node('ENTl3 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl5 = Node('ENTl5 inVoxel=False  n_exp= NA', parent=ENTl)
ENTl1 = Node('ENTl1 inVoxel=False  n_exp= NA', parent=ENTl)
ENTm = Node('ENTm inVoxel=True  n_exp= NA', parent=ENT)
ENTm1 = Node('ENTm1 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm2 = Node('ENTm2 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm3 = Node('ENTm3 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm5 = Node('ENTm5 inVoxel=False  n_exp= NA', parent=ENTm)
ENTm6 = Node('ENTm6 inVoxel=False  n_exp= NA', parent=ENTm)
ENTmv = Node('ENTmv inVoxel=False  n_exp= NA', parent=ENT)
POST = Node('POST inVoxel=True  n_exp= NA', parent=RHP)
PRE = Node('PRE inVoxel=True  n_exp= NA', parent=RHP)
ProS = Node('ProS inVoxel=False  n_exp= NA', parent=RHP)
HATA = Node('HATA inVoxel=False  n_exp= NA', parent=RHP)
HIP = Node('HIP inVoxel=False  n_exp= NA', parent=HPF)
IG = Node('IG inVoxel=True  n_exp= NA', parent=HIP)
CA = Node('CA inVoxel=False  n_exp= NA', parent=HIP)
CA1 = Node('CA1 inVoxel=True  n_exp= NA', parent=CA)
CA2 = Node('CA2 inVoxel=True  n_exp= NA', parent=CA)
CA3 = Node('CA3 inVoxel=True  n_exp= NA', parent=CA)
DG = Node('DG inVoxel=True  n_exp= NA', parent=HIP)
DG_po = Node('DG_po inVoxel=False  n_exp= NA', parent=DG)
DG_sg = Node('DG_sg inVoxel=False  n_exp= NA', parent=DG)
DG_mo = Node('DG_mo inVoxel=False  n_exp= NA', parent=DG)
FC = Node('FC inVoxel=True  n_exp= NA', parent=HIP)
CTXsp = Node('CTXsp inVoxel=False  n_exp= NA', parent=CTX)
LA = Node('LA inVoxel=True  n_exp= NA', parent=CTXsp)
BLA = Node('BLA inVoxel=True  n_exp= NA', parent=CTXsp)
BLAa = Node('BLAa inVoxel=False  n_exp= NA', parent=BLA)
BLAp = Node('BLAp inVoxel=False  n_exp= NA', parent=BLA)
BLAv = Node('BLAv inVoxel=False  n_exp= NA', parent=BLA)
BMA = Node('BMA inVoxel=True  n_exp= NA', parent=CTXsp)
BMAa = Node('BMAa inVoxel=False  n_exp= NA', parent=BMA)
BMAp = Node('BMAp inVoxel=False  n_exp= NA', parent=BMA)
CLA = Node('CLA inVoxel=True  n_exp= NA', parent=CTXsp)
PA = Node('PA inVoxel=True  n_exp= NA', parent=CTXsp)
EP = Node('EP inVoxel=False  n_exp= NA', parent=CTXsp)
EPd = Node('EPd inVoxel=True  n_exp= NA', parent=EP)
EPv = Node('EPv inVoxel=True  n_exp= NA', parent=EP)

In [58]:
for pre, fill, node in RenderTree(CTX):
    print   pre,node.name

 CTX inVoxel=False
├──  CTXpl inVoxel=False  n_exp= NA
│   └──  Isocortex inVoxel=False  n_exp= NA
│       ├──  PTLp inVoxel=False  n_exp= NA
│       │   ├──  VISa inVoxel=True  n_exp= NA
│       │   │   ├──  VISa1 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa2and3 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa4 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa5 inVoxel=False  n_exp= NA
│       │   │   ├──  VISa6a inVoxel=False  n_exp= NA
│       │   │   └──  VISa6b inVoxel=False  n_exp= NA
│       │   └──  VISrl inVoxel=True  n_exp= NA
│       │       ├──  VISrl1 inVoxel=False  n_exp= NA
│       │       ├──  VISrl2and3 inVoxel=False  n_exp= NA
│       │       ├──  VISrl4 inVoxel=False  n_exp= NA
│       │       ├──  VISrl5 inVoxel=False  n_exp= NA
│       │       ├──  VISrl6a inVoxel=False  n_exp= NA
│       │       └──  VISrl6b inVoxel=False  n_exp= NA
│       ├──  ACA inVoxel=False  n_exp= NA
│       │   ├──  ACAd inVoxel=True  n_exp= NA
│       │   │   ├──  ACAd2and3 inVo

### Calculating information flow / number of axons

Since Voxel_Structures only contains structures at the subdivision level, it is not straight forward to get the information flow from large structures to a large structures directly from the Voxel_connectivity. 


In [36]:
def Voxel_infoflow(src,tg,dens,avgfir,avgenc,VMacr_to_error,ddens=46000.0,davgfir=1.0,davgenc=0.5,return_mode='raw',reverse=False,print_Latex_Table=False):
    """Calculates the information flow between each element of the source and target lists.
    
    args:
    src (dict): dictionary of cardinality 1. maps the structure (super) name to its nuclei
    tg (dict): same as src for target
    dens (int): number of neurons per volume (neurons/mm^3)
    avgfir (float): neuron average firing rate per second (firing/s)
    avgenc (float): average number of bits encoded per action potential(bit)
    davgfir (float): error in avgfir
    davgenc (float): error in avgenc
    VMacr_to_error (dict): map from acronym of VM superstructures to the relative error. (for source)
    
    return_mode (str): choose return mode: 'raw','source nuclei to target','source to target'. Default='raw'
    print_Latex_Table (bool): print the formatted table usable for Latex if true. Default=False
    
    returns:
        dictionary containing projections from source to target. Dependant on return_mode
    """
    
    src_name=src.keys()[0] # fetch name of the source. Necessary since the original src is a dictionary
    tg_name=tg.keys()[0] # fetch name of the target
    src=src.values()[0] # list of source acronyms
    tg=tg.values()[0] #list of target acronyms
    

    proj_dict={}
    proj_error_dict={}
    for a in src:
        if a not in proj_dict:
            proj_dict[a]={}
            proj_error_dict[a]={}

        for b in tg:
            proj_value=Voxel_connectivity_dict[('ipsi',a)][b]*dens*avgfir*avgenc
            proj_dict[a][b]=proj_value

            proj_error_value=calc_MSE_error(a,VMacr_to_error,dens,avgfir,avgenc,ddens,davgfir,davgenc)
            proj_error_dict[a][b]=proj_value*proj_error_value
            
    # mode 1: raw
    if return_mode=='raw':
        if print_Latex_Table:
            printLatexTable(proj_dict,proj_error_dict)
        return proj_dict,proj_error_dict
    
    # mode 2" source nuclei to target
    elif return_mode=='source nuclei to target':
        proj_dict_mode2={}
        proj_error_dict_mode2={}
        for a in proj_dict:
            proj_dict_mode2[a]={}
            proj_error_dict_mode2[a]={}
            proj_value=0.0
            proj_error_value=0.0
            for b in proj_dict[a]:
                proj_value+=proj_dict[a][b]
                proj_error_value+=proj_error_dict[a][b]
            proj_dict_mode2[a][tg_name]=proj_value
            proj_error_dict_mode2[a][tg_name]=proj_error_value
        
        if print_Latex_Table:
            printLatexTable(proj_dict_mode2,proj_error_dict_mode2)
            
        return proj_dict_mode2,proj_error_dict_mode2  
    
    
    # return mode 4: source to target nuclei
    elif return_mode=='source to target nuclei':
        proj_dict_mode4={}
        proj_error_dict_mode4={}
        proj_dict_mode4[src_name]={}
        proj_error_dict_mode4[src_name]={}
        for b in tg:
            proj_dict_mode4[src_name][b]=0.0
            proj_error_dict_mode4[src_name][b]=0.0
        
        for a in tg:
            proj_value=0.0
            proj_error_value=0.0
            for b in src:
                proj_value+=proj_dict[b][a]
                proj_error_value+=proj_error_dict[b][a]
            proj_dict_mode4[src_name][a]=proj_value
            proj_error_dict_mode4[src_name][a]=proj_error_value
        
        if print_Latex_Table:
            printLatexTable(proj_dict_mode4,proj_error_dict_mode4)
            
        return proj_dict_mode4,proj_error_dict_mode4    
        
        
    # return mode 3: whole source to whole target
    elif return_mode=='source to target':
        proj_value=0.0
        proj_error_value=0.0
        for a in proj_dict:
            for b in proj_dict[a]:
                proj_value+=proj_dict[a][b]
                proj_error_value+=proj_error_dict[a][b]
        proj_dict_mode3={src_name:{tg_name:proj_value}}
        proj_error_dict_mode3={src_name:{tg_name:proj_error_value}}
                               
        if print_Latex_Table:
            printLatexTable(proj_dict_mode3,proj_error_dict_mode3)
        return proj_dict_mode3,proj_error_dict_mode3
    
    else:
        raise ValueError("Specify an acceptable return mode ('raw','source nuclei to target','source to target')")
    

In [37]:
def calc_MSE_error(src_acr,VMacr_to_error,dens,avgfir,avgenc,ddens,davgfir,davgenc):
    """
    args:
        src_acr (str): acronym of the source structure to calculate uncertainty of
        Voxel_superStructure_error(dict: list of superstructures with uncertainty given by the high resoluton paper
        davgfir (float): error in avgfir
        davgenc (float): error in davgenc
        """
    
    belong=which_do_I_belong(src_acr,VMacr_to_error)
    MSEerror=np.sqrt((VMacr_to_error[belong]/100)**2+(ddens/dens)**2+(davgfir/avgfir)**2+(davgenc/avgfir)**2)
    return MSEerror

In [38]:
def which_do_I_belong(src_acr,VMacr_to_error):
    """calculates whether a structure region is in the superstructure
    args:
        src_arc (str): acronym of the source structure
        superstructure (str): acronym of the super structure
    returns:
        acronym of the structure in which src_acr belongs to"""
    VMacr_superstructures=VMacr_to_error.keys()
    for i in VMacr_superstructures:
        if acronym_to_id[src_acr] in [j for j in Unpack(acronym_to_id[i],Substr,include_self=True)]:
            return i
        
    raise ValueError("No structure found. Check input")


In [39]:
def printLatexTable(proj_dict,proj_error_dict,onlyprint=False):
    """Used to print out lines of code compatible with LaTeX"""
    import math
    
    proj_dict_keys=proj_dict.keys()[0]
    proj_error_dict_keys=proj_error_dict.keys()[0]
    
    if proj_dict.keys()!=proj_error_dict.keys():
        raise ValueError("keys of proj_dict and proj_error_dict not the same.")
    
    for a in proj_dict:
        for b in proj_dict[a]:
            proj_value=proj_dict[a][b]
            spv=str(int(proj_value))
            leng=len(spv)-1
            pspv=spv[0]+"."+spv[1]+"$"+"10^"+str(leng)+"$"
            
            proj_error_value=proj_error_dict[a][b]
            #espv=str(int(proj_error_value))
            #eleng=len(espv)-1
            #epspv=espv[0]+"."+espv[1]+"$"+"10^"+str(eleng)+"$"
            #print a+" & $\\rightarrow$ &"+b+" & "+str(int(proj_value))+" & "+str(int(100*proj_error_value/proj_value))+" \\\\"
            print a+" & $\\rightarrow$ &"+b+" & "+pspv+"&"+str(int(100*proj_error_value/proj_value))+" \\\\"

In [40]:
Voxel_Model_Table1_Superstructures=['Isocortex','Olfactory areas','Cortical subplate',
                                    'Hippocampal region','Striatum','Pallidum','Thalamus',
                                    'Hypothalamus','Midbrain','Pons','Medulla','Cerebellum']

Voxel_Model_Table1_RegionMSErel=[float(i) for i in [33,41,104,56,46,71,59,28,30,52,34,104]]
VMacr_to_error={}

for i in range(len(Voxel_Model_Table1_RegionMSErel)):
    VMacr_to_error[name_to_acronym[Voxel_Model_Table1_Superstructures[i]]]=Voxel_Model_Table1_RegionMSErel[i]

In [41]:
BS_nuclei_Damasio={'BS-d':['SCs',
                   'SCm',
                   'PB',
                   'NTS',
                   'PAG',
                   'AP']}

BS_nuclei={'BS':[id_to_acronym[i] for i in Unpack(acronym_to_id['BS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

TH_nuclei={'TH':[id_to_acronym[i] for i in Unpack(acronym_to_id['TH'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

HY_nuclei={'HY':[id_to_acronym[i] for i in Unpack(acronym_to_id['HY'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

CTX_nuclei={'CTX':[id_to_acronym[i] for i in Unpack(acronym_to_id['CTX'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

Iso_nuclei={'ISO':[id_to_acronym[i] for i in Unpack(acronym_to_id['Isocortex'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

ILM_nuclei={'ILM':[id_to_acronym[i] for i in Unpack(acronym_to_id['ILM'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}


#vision
VIS_nuclei={'VIS':[id_to_acronym[i] for i in Unpack(acronym_to_id['VIS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#hearing
AUD_nuclei={'AUD':[id_to_acronym[i] for i in Unpack(acronym_to_id['AUD'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#somatosensory (touch)
SS_nuclei={'SS':[id_to_acronym[i] for i in Unpack(acronym_to_id['SS'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#taste
GU_nuclei={'GU':[id_to_acronym[i] for i in Unpack(acronym_to_id['GU'],Substr,include_self=True)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
#smell
OLF_nuclei={'OLF':[id_to_acronym[i] for i in Unpack(acronym_to_id['OLF'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

DORsm_nuclei={'DORsm':[id_to_acronym[i] for i in Unpack(acronym_to_id['DORsm'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
DORpm_nuclei={'DORpm':[id_to_acronym[i] for i in Unpack(acronym_to_id['DORpm'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
ILM_nuclei={'ILM':[id_to_acronym[i] for i in Unpack(acronym_to_id['ILM'],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}

SCm_nuclei={'SCm':['SCm']}
SCs_nuclei={'SCs':['SCs']}

Sensory_Cortex=[VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]

In [53]:
def printLatexTable_axon_and_info(proj_dict,proj_error_dict,iproj_dict,iproj_error_dict,onlyprint=False):
    """Used to print out lines of code compatible with LaTeX"""
        
    proj_dict_keys=proj_dict.keys()[0]
    proj_error_dict_keys=proj_error_dict.keys()[0]
    
    if proj_dict.keys()!=proj_error_dict.keys():
        raise ValueError("keys of proj_dict and proj_error_dict not the same.")
    
    for a in proj_dict:
        for b in proj_dict[a]:
            proj_value=proj_dict[a][b]
            spv=str(int(proj_value))
            leng=len(spv)-1
            pspv="$"+spv[0]+"."+spv[1]+"\\times "+"10^"+str(leng)+"$"
            proj_error_value=proj_error_dict[a][b]
            
            iproj_value=iproj_dict[a][b]
            ispv=str(int(iproj_value))
            ileng=len(ispv)-1
            ipspv="$"+str(int(np.floor(float(ispv[0]+"."+ispv[1]))))+"\\times "+"10^"+str(ileng-1)+" - "+str(int(np.ceil(float(ispv[0]+"."+ispv[1]))))+"\\times "+"10^"+str(ileng+1)+"$"
            iproj_error_value=iproj_error_dict[a][b]
            print a+" & $\\rightarrow$ &"+b+" & "+pspv+" & "+str(int(100*proj_error_value/proj_value))+ " & " +ipspv+"\\\\"

### SCs and SCm to sensory cortex

In [54]:
for i in Sensory_Cortex:
    a,b=Voxel_infoflow(SCm_nuclei,i,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    c,d=Voxel_infoflow(SCm_nuclei,i,92000.0,1.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    printLatexTable_axon_and_info(a,b,c,d)

SCm & $\rightarrow$ &VIS & $4.2\times 10^8$ & 58 & $6\times 10^7 - 7\times 10^9$\\
SCm & $\rightarrow$ &AUD & $2.5\times 10^8$ & 58 & $3\times 10^7 - 4\times 10^9$\\
SCm & $\rightarrow$ &SS & $8.5\times 10^8$ & 58 & $1\times 10^8 - 2\times 10^10$\\
SCm & $\rightarrow$ &GU & $4.0\times 10^6$ & 58 & $6\times 10^5 - 7\times 10^7$\\
SCm & $\rightarrow$ &OLF & $1.9\times 10^6$ & 58 & $2\times 10^5 - 3\times 10^7$\\


In [135]:
for i in Sensory_Cortex:
    a,b=Voxel_infoflow(i,SCm_nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    c,d=Voxel_infoflow(i,SCm_nuclei,92000.0,5.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    printLatexTable_axon_and_info(a,b,c,d)

VIS & $\rightarrow$ &SCm & 2.7$\times 10^6$&59 & 2.0$\times 10^7$ & 66\\
AUD & $\rightarrow$ &SCm & 2.0$\times 10^6$&59 & 1.5$\times 10^7$ & 66\\
SS & $\rightarrow$ &SCm & 3.2$\times 10^6$&59 & 2.4$\times 10^7$ & 66\\
GU & $\rightarrow$ &SCm & 2.4$\times 10^5$&59 & 1.8$\times 10^6$ & 66\\
OLF & $\rightarrow$ &SCm & 2.6$\times 10^6$&64 & 1.9$\times 10^7$ & 70\\


In [136]:
for i in Sensory_Cortex:
    a,b=Voxel_infoflow(i,SCs_nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    c,d=Voxel_infoflow(i,SCs_nuclei,92000.0,5.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    printLatexTable_axon_and_info(a,b,c,d)

VIS & $\rightarrow$ &SCs & 1.6$\times 10^6$&59 & 1.2$\times 10^7$ & 66\\
AUD & $\rightarrow$ &SCs & 2.1$\times 10^5$&59 & 1.6$\times 10^6$ & 66\\
SS & $\rightarrow$ &SCs & 1.1$\times 10^6$&59 & 8.9$\times 10^6$ & 66\\
GU & $\rightarrow$ &SCs & 1.1$\times 10^5$&59 & 8.4$\times 10^5$ & 66\\
OLF & $\rightarrow$ &SCs & 1.1$\times 10^6$&64 & 8.3$\times 10^6$ & 70\\


In [137]:
for i in Sensory_Cortex:
    a,b=Voxel_infoflow(SCs_nuclei,i,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    c,d=Voxel_infoflow(SCs_nuclei,i,92000.0,5.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
    printLatexTable_axon_and_info(a,b,c,d)

SCs & $\rightarrow$ &VIS & 3.3$\times 10^8$&58 & 2.5$\times 10^9$ & 64\\
SCs & $\rightarrow$ &AUD & 3.1$\times 10^7$&58 & 2.3$\times 10^8$ & 64\\
SCs & $\rightarrow$ &SS & 2.2$\times 10^7$&58 & 1.6$\times 10^8$ & 64\\
SCs & $\rightarrow$ &GU & 1.0$\times 10^6$&58 & 7.5$\times 10^6$ & 64\\
SCs & $\rightarrow$ &OLF & 2.5$\times 10^6$&58 & 1.9$\times 10^7$ & 64\\


### Thalamic nuclei to sensory and ILM

In [142]:
for i in Sensory_Cortex:
    abc=Voxel_infoflow(i,DORsm_nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in Sensory_Cortex:
    abc=Voxel_infoflow(DORsm_nuclei,i,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)

VIS & $\rightarrow$ &DORsm & 637684118 $\pm$ 382025480 & 59 \\
AUD & $\rightarrow$ &DORsm & 257599140 $\pm$ 154323171 & 59 \\
SS & $\rightarrow$ &DORsm & 1490028676 $\pm$ 892650301 & 59 \\
GU & $\rightarrow$ &DORsm & 30237973 $\pm$ 18115044 & 59 \\
OLF & $\rightarrow$ &DORsm & 17600630 $\pm$ 11380682 & 64 \\
DORsm & $\rightarrow$ &VIS & 424118633 $\pm$ 328000310 & 77 \\
DORsm & $\rightarrow$ &AUD & 308897415 $\pm$ 238891763 & 77 \\
DORsm & $\rightarrow$ &SS & 1914313371 $\pm$ 1480471100 & 77 \\
DORsm & $\rightarrow$ &GU & 65589549 $\pm$ 50724941 & 77 \\
DORsm & $\rightarrow$ &OLF & 13172402 $\pm$ 10187130 & 77 \\


In [143]:
for i in Sensory_Cortex:
    abc=Voxel_infoflow(i,DORpm_nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in Sensory_Cortex:
    abc=Voxel_infoflow(DORpm_nuclei,i,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)

VIS & $\rightarrow$ &DORpm & 634589243 $\pm$ 380171394 & 59 \\
AUD & $\rightarrow$ &DORpm & 213680457 $\pm$ 128012250 & 59 \\
SS & $\rightarrow$ &DORpm & 1187049809 $\pm$ 711140923 & 59 \\
GU & $\rightarrow$ &DORpm & 26222189 $\pm$ 15709258 & 59 \\
OLF & $\rightarrow$ &DORpm & 113785573 $\pm$ 73574494 & 64 \\
DORpm & $\rightarrow$ &VIS & 1349935529 $\pm$ 1043998631 & 77 \\
DORpm & $\rightarrow$ &AUD & 250379881 $\pm$ 193636101 & 77 \\
DORpm & $\rightarrow$ &SS & 1670417127 $\pm$ 1291849244 & 77 \\
DORpm & $\rightarrow$ &GU & 67160597 $\pm$ 51939941 & 77 \\
DORpm & $\rightarrow$ &OLF & 114553036 $\pm$ 88591796 & 77 \\


In [144]:
for i in Sensory_Cortex:
    abc=Voxel_infoflow(ILM_nuclei,i,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)
for i in Sensory_Cortex:
    abc=Voxel_infoflow(i,ILM_nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=True)


ILM & $\rightarrow$ &VIS & 25756904 $\pm$ 19919597 & 77 \\
ILM & $\rightarrow$ &AUD & 4337719 $\pm$ 3354658 & 77 \\
ILM & $\rightarrow$ &SS & 266420532 $\pm$ 206041448 & 77 \\
ILM & $\rightarrow$ &GU & 16222839 $\pm$ 12546245 & 77 \\
ILM & $\rightarrow$ &OLF & 2515250 $\pm$ 1945217 & 77 \\
VIS & $\rightarrow$ &ILM & 7170841 $\pm$ 4295926 & 59 \\
AUD & $\rightarrow$ &ILM & 866369 $\pm$ 519027 & 59 \\
SS & $\rightarrow$ &ILM & 75268351 $\pm$ 45091961 & 59 \\
GU & $\rightarrow$ &ILM & 5610240 $\pm$ 3360997 & 59 \\
OLF & $\rightarrow$ &ILM & 8243786 $\pm$ 5330485 & 64 \\


In [116]:
for i in [VIS_nuclei,AUD_nuclei,SS_nuclei,GU_nuclei,OLF_nuclei]:
    print acronym_to_name[i.keys()[0]]

Visual areas
Auditory areas
Somatosensory areas
Gustatory areas
Olfactory areas


### Between Damasio Nuclei

In [145]:
abc=Voxel_infoflow(BS_nuclei_Damasio,BS_nuclei_Damasio,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='raw',print_Latex_Table=True)

SCm & $\rightarrow$ &SCm & 605901800 $\pm$ 353298425 & 58 \\
SCm & $\rightarrow$ &NTS & 979833 $\pm$ 571336 & 58 \\
SCm & $\rightarrow$ &PB & 8485686 $\pm$ 4947962 & 58 \\
SCm & $\rightarrow$ &AP & 130501 $\pm$ 76094 & 58 \\
SCm & $\rightarrow$ &PAG & 218990470 $\pm$ 127692289 & 58 \\
SCm & $\rightarrow$ &SCs & 256443691 $\pm$ 149531082 & 58 \\
NTS & $\rightarrow$ &SCm & 480307 $\pm$ 290417 & 60 \\
NTS & $\rightarrow$ &NTS & 23296685 $\pm$ 14086309 & 60 \\
NTS & $\rightarrow$ &PB & 1741059 $\pm$ 1052729 & 60 \\
NTS & $\rightarrow$ &AP & 3309791 $\pm$ 2001260 & 60 \\
NTS & $\rightarrow$ &PAG & 636660 $\pm$ 384955 & 60 \\
NTS & $\rightarrow$ &SCs & 14382 $\pm$ 8696 & 60 \\
PB & $\rightarrow$ &SCm & 11463867 $\pm$ 8269891 & 72 \\
PB & $\rightarrow$ &NTS & 8617490 $\pm$ 6216550 & 72 \\
PB & $\rightarrow$ &PB & 36291438 $\pm$ 26180192 & 72 \\
PB & $\rightarrow$ &AP & 550905 $\pm$ 397416 & 72 \\
PB & $\rightarrow$ &PAG & 12451687 $\pm$ 8982492 & 72 \\
PB & $\rightarrow$ &SCs & 856972 $\pm$ 6

### Thalamus nuclei to ISO

In [117]:
%pprint
THsub=['LAT','ATN','MED','ILM','GENv','EPI']
for i in THsub:
    print i+" = "+acronym_to_name[i]

Pretty printing has been turned OFF
LAT = Lateral group of the dorsal thalamus
ATN = Anterior group of the dorsal thalamus
MED = Medial group of the dorsal thalamus
ILM = Intralaminar nuclei of the dorsal thalamus
GENv = Geniculate group, ventral thalamus
EPI = Epithalamus


In [138]:
for k in THsub:
    nuclei={k : [id_to_acronym[i] for i in Unpack(acronym_to_id[k],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
    for p in Sensory_Cortex:
        #Voxel_infoflow(p,nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',print_Latex_Table=True)
        a,b=Voxel_infoflow(p,nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
        c,d=Voxel_infoflow(p,nuclei,92000.0,5.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
        printLatexTable_axon_and_info(a,b,c,d)

VIS & $\rightarrow$ &LAT & 3.9$\times 10^8$&59 & 2.9$\times 10^9$ & 66\\
AUD & $\rightarrow$ &LAT & 1.3$\times 10^8$&59 & 1.0$\times 10^9$ & 66\\
SS & $\rightarrow$ &LAT & 5.0$\times 10^8$&59 & 3.7$\times 10^9$ & 66\\
GU & $\rightarrow$ &LAT & 6.7$\times 10^6$&59 & 5.0$\times 10^7$ & 66\\
OLF & $\rightarrow$ &LAT & 7.3$\times 10^6$&64 & 5.5$\times 10^7$ & 70\\
VIS & $\rightarrow$ &ATN & 7.5$\times 10^7$&59 & 5.6$\times 10^8$ & 66\\
AUD & $\rightarrow$ &ATN & 1.6$\times 10^6$&59 & 1.2$\times 10^7$ & 66\\
SS & $\rightarrow$ &ATN & 9.4$\times 10^7$&59 & 7.1$\times 10^8$ & 66\\
GU & $\rightarrow$ &ATN & 6.4$\times 10^5$&59 & 4.8$\times 10^6$ & 66\\
OLF & $\rightarrow$ &ATN & 1.4$\times 10^7$&64 & 1.0$\times 10^8$ & 70\\
VIS & $\rightarrow$ &MED & 4.2$\times 10^6$&59 & 3.2$\times 10^7$ & 66\\
AUD & $\rightarrow$ &MED & 4.1$\times 10^5$&59 & 3.1$\times 10^6$ & 66\\
SS & $\rightarrow$ &MED & 2.8$\times 10^7$&59 & 2.1$\times 10^8$ & 66\\
GU & $\rightarrow$ &MED & 9.2$\times 10^6$&59 & 6.9$\tim

In [139]:
for k in THsub:
    nuclei={k : [id_to_acronym[i] for i in Unpack(acronym_to_id[k],Substr)
          if i in [acronym_to_id[j] for j in Voxel_Structures]]}
    for p in Sensory_Cortex:
        #Voxel_infoflow(p,nuclei,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',print_Latex_Table=True)
        a,b=Voxel_infoflow(nuclei,p,92000.0,1.0,1.0,VMacr_to_error,davgfir=0.0,davgenc=0.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
        c,d=Voxel_infoflow(nuclei,p,92000.0,5.0,1.5,VMacr_to_error,davgfir=1.0,davgenc=1.0,return_mode='source to target',reverse=False,print_Latex_Table=False)
        printLatexTable_axon_and_info(a,b,c,d)

LAT & $\rightarrow$ &VIS & 5.8$\times 10^8$&77 & 4.3$\times 10^9$ & 82\\
LAT & $\rightarrow$ &AUD & 1.6$\times 10^8$&77 & 1.2$\times 10^9$ & 82\\
LAT & $\rightarrow$ &SS & 8.8$\times 10^8$&77 & 6.6$\times 10^9$ & 82\\
LAT & $\rightarrow$ &GU & 1.6$\times 10^7$&77 & 1.2$\times 10^8$ & 82\\
LAT & $\rightarrow$ &OLF & 8.6$\times 10^5$&77 & 6.4$\times 10^6$ & 82\\
ATN & $\rightarrow$ &VIS & 3.7$\times 10^8$&77 & 2.8$\times 10^9$ & 82\\
ATN & $\rightarrow$ &AUD & 1.1$\times 10^7$&77 & 8.4$\times 10^7$ & 82\\
ATN & $\rightarrow$ &SS & 8.3$\times 10^7$&77 & 6.2$\times 10^8$ & 82\\
ATN & $\rightarrow$ &GU & 7.5$\times 10^5$&77 & 5.6$\times 10^6$ & 82\\
ATN & $\rightarrow$ &OLF & 4.6$\times 10^6$&77 & 3.4$\times 10^7$ & 82\\
MED & $\rightarrow$ &VIS & 1.2$\times 10^7$&77 & 9.2$\times 10^7$ & 82\\
MED & $\rightarrow$ &AUD & 1.9$\times 10^6$&77 & 1.4$\times 10^7$ & 82\\
MED & $\rightarrow$ &SS & 8.7$\times 10^7$&77 & 6.5$\times 10^8$ & 82\\
MED & $\rightarrow$ &GU & 2.2$\times 10^7$&77 & 1.6$\tim